In [1]:
import pandas as pd
import psycopg2
import datetime
import numpy as np
import random
import matplotlib.pyplot as plt

In [2]:
# # Connect to the PostgreSQL database
# def connect_database():
#     conn = psycopg2.connect(
#         host="127.0.0.1",
#         port="5432",
#         database="cognitive_battery",
#         user="postgres",
#         password="postgres"
#     )
#     return conn

# # Execute an SQL query and fetch data into a DataFrame
# def fetch_data(conn, query):
#     try:
#         # Use the connection to execute the query
#         cursor = conn.cursor()
#         cursor.execute(query)
        
#         # Fetch all the rows into a DataFrame
#         data = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
        
#         return data
#     except Exception as e:
#         print(f"Error fetching data: {e}")

In [3]:
# connection = connect_database()

# query1 = "SELECT subject_id, eye_data, game_type FROM public.eyes_data"
# df_eye = fetch_data(connection, query1)
# print("Finished query1")

# # Fetch data from the second table (public.sensor_data)
# query2 = "SELECT subject_id, data, game_type FROM public.sensor_data"
# df_sensor = fetch_data(connection, query2)
# print("Finished query2")

# # Fetch data from the third table (public.task_switching)
# query3 = "SELECT userid, data FROM public.task_switching"
# df_task_switch = fetch_data(connection, query3)
# print("Finished query3")

# # Fetch data from the fourth table (public.working_memory)
# query4 = "SELECT userid, data FROM public.working_memory"
# df_working_memory = fetch_data(connection, query4)
# print("Finished query4")

# # Fetch data from the fourth table (public.working_memory)
# query5 = "SELECT userid, data FROM public.enumeration"
# df_enumeration= fetch_data(connection, query5)

In [4]:
# # Save df_eye
# df_eye.to_pickle('df_eye.pkl')

# # Save df_sensor
# df_sensor.to_pickle('df_sensor.pkl')

# # Save df_task_switch
# df_task_switch.to_pickle('df_task_switch.pkl')

# # Save df_working_memory
# df_working_memory.to_pickle('df_working_memory.pkl')

# # Save df_enumeration
# df_enumeration.to_pickle('df_enumeration.pkl')

# READ files from pkl

In [5]:
# Load df_eye
df_eye = pd.read_pickle('df_eye.pkl')

# Load df_sensor
df_sensor = pd.read_pickle('df_sensor.pkl')

# Load df_task_switch
df_task_switch = pd.read_pickle('df_task_switch.pkl')

# Load df_working_memory
df_working_memory = pd.read_pickle('df_working_memory.pkl')

# Load df_enumeration
df_enumeration = pd.read_pickle('df_enumeration.pkl')

In [6]:
def convert_timestamp_column(df, column_name):
    # Convert the column to numeric (assuming the values are in string format)
    df[column_name] = pd.to_numeric(df[column_name], errors='coerce')  # 'coerce' converts non-numeric values to NaN
    
    # Drop rows with NaN values (if needed)
    df.dropna(subset=[column_name], inplace=True)
    
    # Convert the Unix timestamp to a datetime object
    df['Datetime'] = pd.to_datetime(df[column_name], unit='s')
    
    # Extract month, day, hour, minute, and second
    df['Month'] = df['Datetime'].dt.strftime('%m')
    df['Day'] = df['Datetime'].dt.strftime('%d')
    df['Hour'] = df['Datetime'].dt.strftime('%H')
    df['Minute'] = df['Datetime'].dt.strftime('%M')
    df['Second'] = df['Datetime'].dt.strftime('%S')
    
    # Replace the original column with the new datetime column
    df[column_name] = df['Datetime']
    
    # Drop the 'Datetime' column if our no longer need it
    df.drop(columns=['Datetime'], inplace=True)
    
    return df

# Eye data

In [7]:
df_eye.head(10)

,subject_id,eye_data,game_type
0,BVCX,"{'Timestamp': [1710352642777.923, 171035264276...",baseline
1,BVCX,"{'Timestamp': [1710354603794.525, 171035460381...",enumeration
2,BVCX,"{'Timestamp': [1710354023688.477, 171035402375...",task_switching
3,BVCX,"{'Timestamp': [1710353127527.266, 171035312754...",working_memory
4,CVBN,"{'Timestamp': [1710444817133.011, 171044481714...",baseline
5,CVBN,"{'Timestamp': [1710445670282.851, 171044567029...",enumeration
6,CVBN,"{'Timestamp': [1710446344021.539, 171044634403...",task_switching
7,CVBN,"{'Timestamp': [1710447091860.531, 171044709187...",working_memory
8,DKLQ,"{'Timestamp': [1707850429565.024, 170785042957...",baseline
9,DKLQ,"{'Timestamp': [1707850834819.694, 170785083487...",enumeration


In [8]:
# Create a function to extract and flatten the dictionaries
def extract_dict(row, key):
    if key in row:
        return row[key]
    else:
        return None

In [9]:
# Extract and flatten each dictionary key into separate columns
for key in df_eye ['eye_data'].iloc[0].keys():
    df_eye[key] = df_eye['eye_data'].apply(lambda x: extract_dict(x, key))

# Drop the original 'eye_data' column
df_eye.drop(columns='eye_data', inplace=True)

In [10]:
# Define a function to handle 'None' values and convert to NumPy array
def handle_none_and_convert_to_numpy(data):
    cleaned_data = [np.nan if val == 'None' else float(val) for val in data]
    return np.array(cleaned_data)

# Apply the function to the relevant columns in df1
df_eye['Timestamp'] = df_eye['Timestamp'].apply(handle_none_and_convert_to_numpy)
df_eye['Gaze_X'] = df_eye['Gaze_X'].apply(handle_none_and_convert_to_numpy)
df_eye['Gaze_Y'] = df_eye['Gaze_Y'].apply(handle_none_and_convert_to_numpy)
df_eye['Pupil_left'] = df_eye['Pupil_left'].apply(handle_none_and_convert_to_numpy)
df_eye['Pupil_right'] = df_eye['Pupil_right'].apply(handle_none_and_convert_to_numpy)

In [11]:
df_eye["subject_id"].unique()

array(['BVCX', 'CVBN', 'DKLQ', 'EDFR', 'FDSA', 'FGHZ', 'HGFD', 'JHGF',
       'JKLZ', 'MYTZ', 'NMBV', 'NMKL', 'PKJH', 'PLMO', 'QPMN', 'QWER',
       'QWRT', 'QXZW', 'SBQRG', 'SJWQP', 'SVTZM', 'SXVBN', 'TGVB', 'TYDF',
       'WERT', 'WZXE', 'XZPT', 'YUZW', 'ZXAS', 'ZXWE', 'ZYXW', 'BHGY',
       'HJKL', 'JFDE', 'JKLM', 'KJHG', 'LKJH', 'LPDW', 'MJKL', 'MJUY',
       'NBVC', 'NVBX', 'QAXY', 'QPLM', 'REWQ', 'RIQW', 'RTGH', 'SBTBT',
       'SNMVN', 'TYZX', 'UIOP', 'UYGH', 'VIWZ', 'VNPL', 'XCVH', 'XRTY',
       'ZRTY', 'ZXCJ', 'ZXCV', 'ZXNM', 'FTLD', 'FYDR', 'GHJK', 'GTYU',
       'HPLX', 'JKZX', 'LKMN', 'LKPW', 'LPZV', 'NHZA', 'NMJU', 'POIU',
       'STWXJ', 'UJNH', 'UPLK', 'UYTR', 'VBNM', 'XCVN', 'AXSW', 'DFGH',
       'DFZX', 'FDXC', 'FTYP', 'GHFJ', 'LRTB', 'MNBW', 'MNOP', 'MTYH',
       'OPKJ', 'PLRQ', 'QAZW', 'QPOI', 'RQWE', 'SHYXZ', 'SLMJK', 'SPKZR',
       'SRKLP', 'VFTY', 'XUYP', 'YTRE'], dtype=object)

In [12]:
df_eye.head()

,subject_id,game_type,Timestamp,Gaze_X,Gaze_Y,Pupil_left,Pupil_right
0,BVCX,baseline,"[1710352642777.923, 1710352642769.658, 1710352...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, 2.95977783203125, 2.990264892578125..."
1,BVCX,enumeration,"[1710354603794.525, 1710354603817.258, 1710354...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
2,BVCX,task_switching,"[1710354023688.477, 1710354023752.989, 1710354...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
3,BVCX,working_memory,"[1710353127527.266, 1710353127549.055, 1710353...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
4,CVBN,baseline,"[1710444817133.011, 1710444817145.584, 1710444...","[nan, 0.5115212798118591, 0.5106108784675598, ...","[nan, 0.5157326459884644, 0.5222259759902954, ...","[nan, 3.527740478515625, 3.4796142578125, 3.49...","[nan, 3.4715728759765625, 3.43231201171875, 3...."


In [13]:
# Create an empty list to store the datetime values
date_values = []

# Iterate through each row and process the lists of timestamps
for timestamp_list in df_eye['Timestamp']:
    # Initialize an empty list to store the converted timestamps for this row
    row_dates = []
    
    # Process each Unix timestamp in the list and convert to datetime
    for timestamp in timestamp_list:
        date = pd.to_datetime(timestamp, unit='ms')
        row_dates.append(date)
    
    # Append the list of datetime values for this row to date_values
    date_values.append(row_dates)

# Create a new column in the DataFrame to store the datetime values
df_eye['Dates'] = date_values

In [14]:
df_eye.head()

,subject_id,game_type,Timestamp,Gaze_X,Gaze_Y,Pupil_left,Pupil_right,Dates
0,BVCX,baseline,"[1710352642777.923, 1710352642769.658, 1710352...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, 2.95977783203125, 2.990264892578125...","[2024-03-13 17:57:22.777923072, 2024-03-13 17:..."
1,BVCX,enumeration,"[1710354603794.525, 1710354603817.258, 1710354...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2024-03-13 18:30:03.794524928, 2024-03-13 18:..."
2,BVCX,task_switching,"[1710354023688.477, 1710354023752.989, 1710354...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2024-03-13 18:20:23.688476928, 2024-03-13 18:..."
3,BVCX,working_memory,"[1710353127527.266, 1710353127549.055, 1710353...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2024-03-13 18:05:27.527266048, 2024-03-13 18:..."
4,CVBN,baseline,"[1710444817133.011, 1710444817145.584, 1710444...","[nan, 0.5115212798118591, 0.5106108784675598, ...","[nan, 0.5157326459884644, 0.5222259759902954, ...","[nan, 3.527740478515625, 3.4796142578125, 3.49...","[nan, 3.4715728759765625, 3.43231201171875, 3....","[2024-03-14 19:33:37.133010944, 2024-03-14 19:..."


In [15]:
# Define a function to extract the first date from a list of datetime values
def extract_first_date(date_list):
    if isinstance(date_list, list) and len(date_list) > 0:
        return date_list[0].date()
    else:
        return None

In [16]:
df_eye['Date'] = df_eye['Dates'].apply(extract_first_date)

In [17]:
df_eye.head()

,subject_id,game_type,Timestamp,Gaze_X,Gaze_Y,Pupil_left,Pupil_right,Dates,Date
0,BVCX,baseline,"[1710352642777.923, 1710352642769.658, 1710352...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, 2.95977783203125, 2.990264892578125...","[2024-03-13 17:57:22.777923072, 2024-03-13 17:...",2024-03-13
1,BVCX,enumeration,"[1710354603794.525, 1710354603817.258, 1710354...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2024-03-13 18:30:03.794524928, 2024-03-13 18:...",2024-03-13
2,BVCX,task_switching,"[1710354023688.477, 1710354023752.989, 1710354...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2024-03-13 18:20:23.688476928, 2024-03-13 18:...",2024-03-13
3,BVCX,working_memory,"[1710353127527.266, 1710353127549.055, 1710353...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2024-03-13 18:05:27.527266048, 2024-03-13 18:...",2024-03-13
4,CVBN,baseline,"[1710444817133.011, 1710444817145.584, 1710444...","[nan, 0.5115212798118591, 0.5106108784675598, ...","[nan, 0.5157326459884644, 0.5222259759902954, ...","[nan, 3.527740478515625, 3.4796142578125, 3.49...","[nan, 3.4715728759765625, 3.43231201171875, 3....","[2024-03-14 19:33:37.133010944, 2024-03-14 19:...",2024-03-14


In [18]:
# Replace values in the "game_type" column
df_eye['game_type'] = df_eye['game_type'].replace({'task_switching': 'task switch', 'working_memory': 'working memory'})

In [19]:
df_eye["game_type"].unique()

array(['baseline', 'enumeration', 'task switch', 'working memory'],
      dtype=object)

# Sensor

In [20]:
df_sensor.head(10)

,subject_id,data,game_type
0,BVCX,"{'Timestamp': [1710352670.22933, 1710352670.23...",baseline
1,BVCX,"{'Timestamp': [1710354601.587721, 1710354601.5...",enumeration
2,BVCX,"{'Timestamp': [1710354051.4275117, 1710354051....",task_switching
3,BVCX,"{'Timestamp': [1710353155.3258922, 1710353155....",working_memory
4,CVBN,"{'Timestamp': [1710444844.91457, 1710444844.91...",baseline
5,CVBN,"{'Timestamp': [1710445698.046139, 1710445698.0...",enumeration
6,CVBN,"{'Timestamp': [1710446371.78589, 1710446371.78...",task_switching
7,CVBN,"{'Timestamp': [1710447119.6544454, 1710447119....",working_memory
8,DKLQ,"{'Timestamp': [1707850457.359229, 1707850457.3...",baseline
9,DKLQ,"{'Timestamp': [1707850862.6135108, 1707850862....",enumeration


In [21]:
# Extract and flatten each dictionary key into separate columns
for key in df_sensor['data'].iloc[0].keys():
    df_sensor[key] = df_sensor['data'].apply(lambda x: extract_dict(x, key))

# Drop the original 'eye_data' column
df_sensor.drop(columns='data', inplace=True)

In [22]:
df_sensor.head()

,subject_id,game_type,Timestamp,ecg_data,eda_data
0,BVCX,baseline,"[1710352670.22933, 1710352670.23333, 171035267...","[210.0, 215.0, 210.0, 203.0, 166.0, 132.0, 133...","[209.0, 211.0, 208.0, 201.0, 166.0, 133.0, 131..."
1,BVCX,enumeration,"[1710354601.587721, 1710354601.591721, 1710354...","[236.0, 239.0, 234.0, 236.0, 237.0, 233.0, 239...","[233.0, 236.0, 232.0, 233.0, 235.0, 232.0, 236..."
2,BVCX,task_switching,"[1710354051.4275117, 1710354051.4315114, 17103...","[180.0, 178.0, 157.0, 155.0, 153.0, 163.0, 156...","[178.0, 176.0, 155.0, 152.0, 151.0, 160.0, 155..."
3,BVCX,working_memory,"[1710353155.3258922, 1710353155.3298922, 17103...","[418.0, 363.0, 272.0, 202.0, 179.0, 157.0, 137...","[416.0, 364.0, 274.0, 203.0, 175.0, 155.0, 137..."
4,CVBN,baseline,"[1710444844.91457, 1710444844.9185698, 1710444...","[207.0, 203.0, 200.0, 196.0, 188.0, 193.0, 186...","[203.0, 202.0, 197.0, 193.0, 187.0, 190.0, 184..."


In [23]:
# Apply the function to the relevant columns in df1
df_sensor['Timestamp'] = df_sensor['Timestamp'].apply(handle_none_and_convert_to_numpy)
df_sensor['ecg_data'] = df_sensor['ecg_data'].apply(handle_none_and_convert_to_numpy)
df_sensor['eda_data'] = df_sensor['eda_data'].apply(handle_none_and_convert_to_numpy)

In [24]:
# Create an empty list to store the datetime values
date_values = []

# Iterate through each row and process the lists of timestamps
for timestamp_list in df_sensor['Timestamp']:
    # Initialize an empty list to store the converted timestamps for this row
    row_dates = []
    
    # Process each Unix timestamp in the list and convert to datetime
    for timestamp in timestamp_list:
        date = pd.to_datetime(timestamp, unit='s')
        row_dates.append(date)
    
    # Append the list of datetime values for this row to date_values
    date_values.append(row_dates)

# Create a new column in the DataFrame to store the datetime values
df_sensor['Dates'] = date_values

In [25]:
df_sensor['Date'] = df_sensor['Dates'].apply(extract_first_date)

In [26]:
df_sensor['game_type'] = df_sensor['game_type'].replace({'task_switching': 'task switch', 'working_memory': 'working memory'})

In [27]:
df_sensor.head()

,subject_id,game_type,Timestamp,ecg_data,eda_data,Dates,Date
0,BVCX,baseline,"[1710352670.22933, 1710352670.23333, 171035267...","[210.0, 215.0, 210.0, 203.0, 166.0, 132.0, 133...","[209.0, 211.0, 208.0, 201.0, 166.0, 133.0, 131...","[2024-03-13 17:57:50.229330176, 2024-03-13 17:...",2024-03-13
1,BVCX,enumeration,"[1710354601.587721, 1710354601.591721, 1710354...","[236.0, 239.0, 234.0, 236.0, 237.0, 233.0, 239...","[233.0, 236.0, 232.0, 233.0, 235.0, 232.0, 236...","[2024-03-13 18:30:01.587721216, 2024-03-13 18:...",2024-03-13
2,BVCX,task switch,"[1710354051.4275117, 1710354051.4315114, 17103...","[180.0, 178.0, 157.0, 155.0, 153.0, 163.0, 156...","[178.0, 176.0, 155.0, 152.0, 151.0, 160.0, 155...","[2024-03-13 18:20:51.427511808, 2024-03-13 18:...",2024-03-13
3,BVCX,working memory,"[1710353155.3258922, 1710353155.3298922, 17103...","[418.0, 363.0, 272.0, 202.0, 179.0, 157.0, 137...","[416.0, 364.0, 274.0, 203.0, 175.0, 155.0, 137...","[2024-03-13 18:05:55.325892096, 2024-03-13 18:...",2024-03-13
4,CVBN,baseline,"[1710444844.91457, 1710444844.9185698, 1710444...","[207.0, 203.0, 200.0, 196.0, 188.0, 193.0, 186...","[203.0, 202.0, 197.0, 193.0, 187.0, 190.0, 184...","[2024-03-14 19:34:04.914569984, 2024-03-14 19:...",2024-03-14


# enumeration

In [69]:
# # Load df_enumeration
# df_enumeration = pd.read_pickle('df_enumeration.pkl')

In [70]:
# Extract and flatten each dictionary key into separate columns
for key in df_enumeration['data'].iloc[0].keys():
    df_enumeration[key] = df_enumeration['data'].apply(lambda x: extract_dict(x, key))

# Drop the original 'eye_data' column
df_enumeration.drop(columns='data', inplace=True)

In [71]:
df_enumeration.head()

,userid,Blockno,Difficulty,Start_time,End_time,Results_Responses,Results_rt,Results_targetvalue,Accuracy,Task_type,Accuracy_Easy,Accuracy_Medium,Accuracy_Hard,Accuracy_Overall
0,BVCX,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1710354640593, 1710354645123, 1710354648903, ...","[1710354642601, 1710354646364, 1710354650563, ...","[8, 8, 9, 6, 5, 7, 6, 7, 6, 5, 5, 5, 9, 7, 8, ...","[2008, 1241, 1660, 1247, 1428, 1614, 2485, 109...","[9, 8, 9, 6, 5, 7, 6, 7, 6, 5, 5, 5, 9, 7, 8, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",Enumeration,0.90,0.55,0.30,0.583333
1,CVBN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Hard, Hard, Hard, Hard, Hard, Hard, Hard, Har...","[1710445737115, 1710445742795, 1710445747935, ...","[1710445740702, 1710445745849, 1710445753169, ...","[7, 5, 7, 6, 9, 6, 5, 8, 7, 6, 6, 7, 5, 6, 8, ...","[3587, 3054, 5234, 2799, 5034, 1613, 2558, 596...","[7, 5, 7, 6, 9, 6, 5, 8, 8, 8, 6, 8, 5, 6, 9, ...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, ...",Enumeration,0.80,0.75,0.70,0.750000
2,DKLQ,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Medium, Medium, Medium, Medium, Medium, Mediu...","[1707850878427, 1707850882387, 1707850886547, ...","[1707850880091, 1707850884259, 1707850888453, ...","[6, 6, 8, 5, 9, 10, 7, 7, 10, 7, 6, 5, 5, 5, 7...","[1664, 1872, 1906, 1130, 3332, 2137, 1609, 189...","[6, 6, 9, 5, 8, 8, 7, 7, 9, 6, 6, 5, 5, 5, 7, ...","[1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, ...",Enumeration,0.90,0.70,0.55,0.716667
3,EDFR,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1710266348279, 1710266352519, 1710266356759, ...","[1710266349996, 1710266354236, 1710266358444, ...","[5, 6, 9, 5, 6, 7, 8, 7, 8, 9, 6, 7, 5, 6, 7, ...","[1717, 1717, 1685, 1711, 1999, 1412, 1659, 162...","[5, 6, 9, 5, 6, 8, 8, 7, 8, 9, 6, 7, 5, 6, 7, ...","[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",Enumeration,0.85,0.80,0.50,0.716667
4,FDSA,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1712166925090, 1712166931190, 1712166935810, ...","[1712166928655, 1712166933286, 1712166939153, ...","[6, 5, 9, 10, 6, 7, 8, 6, 5, 8, 7, 8, 7, 5, 9,...","[3565, 2096, 3343, 2970, 2511, 2391, 2261, 167...","[6, 5, 9, 8, 7, 6, 8, 6, 5, 9, 7, 8, 7, 5, 9, ...","[1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, ...",Enumeration,0.75,0.75,0.65,0.716667


In [72]:
# Assuming we have a DataFrame called df_task_switch
# Rename the "userid" column to "subject_id"
df_enumeration.rename(columns={'userid': 'subject_id'}, inplace=True)

# Rename the "Task_type" column to "game_type"
df_enumeration.rename(columns={'Task_type': 'game_type'}, inplace=True)

In [73]:
df_enumeration['game_type'] = df_enumeration['game_type'].replace({'Enumeration': 'enumeration'})
df_enumeration.rename(columns={'Blockno': 'BlockNo'}, inplace=True)

In [74]:
# Create an empty list to store the datetime values for Start_time
start_time_values = []

# Create an empty list to store the datetime values for End_time
end_time_values = []

# Iterate through each row and process the lists of timestamps for both Start_time and End_time
for start_timestamp_list, end_timestamp_list in zip(df_enumeration['Start_time'], df_enumeration['End_time']):
    # Initialize empty lists to store the converted timestamps for this row
    start_row_dates = []
    end_row_dates = []
    
    # Process each Unix timestamp in the Start_time list and convert to datetime
    for start_timestamp in start_timestamp_list:
        start_date = pd.to_datetime(start_timestamp, unit='ms')
        start_row_dates.append(start_date)
    
    # Process each Unix timestamp in the End_time list and convert to datetime
    for end_timestamp in end_timestamp_list:
        end_date = pd.to_datetime(end_timestamp, unit='ms')
        end_row_dates.append(end_date)
    
    # Append the list of datetime values for Start_time and End_time for this row
    start_time_values.append(start_row_dates)
    end_time_values.append(end_row_dates)

# Create new columns in the DataFrame to store the datetime values for Start_time and End_time
df_enumeration['Start_time_Dates'] = start_time_values
df_enumeration['End_time_Dates'] = end_time_values

In [76]:
df_enumeration['Date'] = df_enumeration['Start_time_Dates'].apply(extract_first_date)

# working_memory

In [35]:
# Extract and flatten each dictionary key into separate columns
for key in df_working_memory['data'].iloc[0].keys():
    df_working_memory[key] = df_working_memory['data'].apply(lambda x: extract_dict(x, key))

# Drop the original 'eye_data' column
df_working_memory.drop(columns='data', inplace=True)

In [36]:
df_working_memory.head()

,userid,BlockNo,Difficulty,Start_time,End_time,Results_responses,Results_rt,Results_targetvalue_stim,Results_num_stim,Results_correct,Correct Answer,Accuracy,Task_type,Accuracy_Easy,Accuracy_Medium,Accuracy_Hard,Accuracy_Overall
0,BVCX,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1710353214025, 1710353222205, 1710353229325, ...","[1710353216085, 1710353224405, 1710353231425, ...","[0-1-2-3, 0-1-2-3, 0-1-2, 0-1-2-3, 0-1-2, 0-1-...","[2060, 2200, 2100, 2720, 1980, 2040, 1761, 164...","[5-11-8-13-2-7-1-14-12-3-10-6-15-9-0-4, 7-3-6-...","[4, 4, 3, 4, 3, 4, 3, 3, 3, 3, 4, 4, 4, 4, 4, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0-1-2-3, 0-1-2-3, 0-1-2, 0-1-2-3, 0-1-2, 0-1-...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",working memory,1.000000,0.833333,0.531250,0.788194
1,CVBN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1710447182643, 1710447189623, 1710447197283, ...","[1710447184703, 1710447191163, 1710447200103, ...","[0-1-2, 0-1-2, 0-1-2-3, 0-1-2, 0-1-2-3, 0-1-2-...","[2060, 1540, 2820, 2120, 2340, 2060, 2660, 242...","[7-4-9-14-15-1-5-12-6-11-0-2-10-8-13-3, 9-5-3-...","[3, 3, 4, 3, 4, 4, 4, 3, 3, 4, 4, 3, 3, 3, 4, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0-1-2, 0-1-2, 0-1-2-3, 0-1-2, 0-1-2-3, 0-1-2-...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",working memory,1.000000,0.896667,0.648214,0.848294
2,DKLQ,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Medium, Medium, Medium, Medium, Medium, Mediu...","[1707852181317, 1707852191218, 1707852201798, ...","[1707852183897, 1707852194478, 1707852203938, ...","[0-1-2-3-4, 0-1-2-3-4, 0-1-2-3-4, 0-1-2-3-4-5,...","[2580, 3260, 2140, 3179, 2779, 2699, 4420, 278...","[15-10-1-3-4-9-12-5-6-2-13-0-14-11-8-7, 0-3-14...","[5, 5, 5, 6, 6, 5, 5, 6, 5, 6, 5, 6, 5, 5, 6, ...","[1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, ...","[0-1-2-3-4, 0-1-2-3-4, 0-1-2-3-4, 0-1-2-3-4-5,...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6, 1.0, 0.6, ...",working memory,1.000000,0.918333,0.513393,0.810575
3,EDFR,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Hard, Hard, Hard, Hard, Hard, Hard, Hard, Har...","[1710266757112, 1710266777110, 1710266792850, ...","[1710266766171, 1710266783130, 1710266795630, ...","[0-1-2-5-11-6-7-4, 0-1-2-3-4-15-7-5, 0-1-3-2-4...","[9059, 6020, 2780, 3340, 6419, 2740, 5540, 300...","[0-11-13-7-5-10-6-15-12-1-3-9-14-2-8-4, 14-12-...","[8, 8, 7, 8, 8, 7, 8, 8, 7, 7, 7, 8, 7, 8, 7, ...","[0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, ...","[0-1-2-3-4-5-6-7, 0-1-2-3-4-5-6-7, 0-1-2-3-4-5...","[0.375, 0.625, 0.7142857142857143, 1.0, 0.5, 1...",working memory,0.987500,0.973333,0.664286,0.875040
4,FDSA,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1712167480231, 1712167488491, 1712167496271, ...","[1712167482371, 1712167490151, 1712167498131, ...","[0-1-2-3, 0-1-2-3, 0-1-2-3, 0-1-2, 0-1-2-3, 0-...","[2140, 1660, 1860, 2140, 2260, 1440, 1660, 232...","[6-8-15-5-0-1-11-12-7-9-3-10-13-2-4-14, 15-14-...","[4, 4, 4, 3, 4, 3, 3, 4, 3, 3, 4, 4, 3, 4, 4, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, ...","[0-1-2-3, 0-1-2-3, 0-1-2-3, 0-1-2, 0-1-2-3, 0-...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",working memory,0.966667,0.855000,0.639286,0.820317


In [37]:
# Create an empty list to store the datetime values for Start_time
start_time_values = []

# Create an empty list to store the datetime values for End_time
end_time_values = []

# Iterate through each row and process the lists of timestamps for both Start_time and End_time
for start_timestamp_list, end_timestamp_list in zip(df_working_memory['Start_time'], df_working_memory['End_time']):
    # Initialize empty lists to store the converted timestamps for this row
    start_row_dates = []
    end_row_dates = []
    
    # Process each Unix timestamp in the Start_time list and convert to datetime
    for start_timestamp in start_timestamp_list:
        start_date = pd.to_datetime(start_timestamp, unit='ms')
        start_row_dates.append(start_date)
    
    # Process each Unix timestamp in the End_time list and convert to datetime
    for end_timestamp in end_timestamp_list:
        end_date = pd.to_datetime(end_timestamp, unit='ms')
        end_row_dates.append(end_date)
    
    # Append the list of datetime values for Start_time and End_time for this row
    start_time_values.append(start_row_dates)
    end_time_values.append(end_row_dates)

# Create new columns in the DataFrame to store the datetime values for Start_time and End_time
df_working_memory['Start_time_Dates'] = start_time_values
df_working_memory['End_time_Dates'] = end_time_values

In [38]:
df_working_memory.head()

,userid,BlockNo,Difficulty,Start_time,End_time,Results_responses,Results_rt,Results_targetvalue_stim,Results_num_stim,Results_correct,Correct Answer,Accuracy,Task_type,Accuracy_Easy,Accuracy_Medium,Accuracy_Hard,Accuracy_Overall,Start_time_Dates,End_time_Dates
0,BVCX,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1710353214025, 1710353222205, 1710353229325, ...","[1710353216085, 1710353224405, 1710353231425, ...","[0-1-2-3, 0-1-2-3, 0-1-2, 0-1-2-3, 0-1-2, 0-1-...","[2060, 2200, 2100, 2720, 1980, 2040, 1761, 164...","[5-11-8-13-2-7-1-14-12-3-10-6-15-9-0-4, 7-3-6-...","[4, 4, 3, 4, 3, 4, 3, 3, 3, 3, 4, 4, 4, 4, 4, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0-1-2-3, 0-1-2-3, 0-1-2, 0-1-2-3, 0-1-2, 0-1-...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",working memory,1.000000,0.833333,0.531250,0.788194,"[2024-03-13 18:06:54.025000, 2024-03-13 18:07:...","[2024-03-13 18:06:56.085000, 2024-03-13 18:07:..."
1,CVBN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1710447182643, 1710447189623, 1710447197283, ...","[1710447184703, 1710447191163, 1710447200103, ...","[0-1-2, 0-1-2, 0-1-2-3, 0-1-2, 0-1-2-3, 0-1-2-...","[2060, 1540, 2820, 2120, 2340, 2060, 2660, 242...","[7-4-9-14-15-1-5-12-6-11-0-2-10-8-13-3, 9-5-3-...","[3, 3, 4, 3, 4, 4, 4, 3, 3, 4, 4, 3, 3, 3, 4, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0-1-2, 0-1-2, 0-1-2-3, 0-1-2, 0-1-2-3, 0-1-2-...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",working memory,1.000000,0.896667,0.648214,0.848294,"[2024-03-14 20:13:02.643000, 2024-03-14 20:13:...","[2024-03-14 20:13:04.703000, 2024-03-14 20:13:..."
2,DKLQ,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Medium, Medium, Medium, Medium, Medium, Mediu...","[1707852181317, 1707852191218, 1707852201798, ...","[1707852183897, 1707852194478, 1707852203938, ...","[0-1-2-3-4, 0-1-2-3-4, 0-1-2-3-4, 0-1-2-3-4-5,...","[2580, 3260, 2140, 3179, 2779, 2699, 4420, 278...","[15-10-1-3-4-9-12-5-6-2-13-0-14-11-8-7, 0-3-14...","[5, 5, 5, 6, 6, 5, 5, 6, 5, 6, 5, 6, 5, 5, 6, ...","[1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, ...","[0-1-2-3-4, 0-1-2-3-4, 0-1-2-3-4, 0-1-2-3-4-5,...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6, 1.0, 0.6, ...",working memory,1.000000,0.918333,0.513393,0.810575,"[2024-02-13 19:23:01.317000, 2024-02-13 19:23:...","[2024-02-13 19:23:03.897000, 2024-02-13 19:23:..."
3,EDFR,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Hard, Hard, Hard, Hard, Hard, Hard, Hard, Har...","[1710266757112, 1710266777110, 1710266792850, ...","[1710266766171, 1710266783130, 1710266795630, ...","[0-1-2-5-11-6-7-4, 0-1-2-3-4-15-7-5, 0-1-3-2-4...","[9059, 6020, 2780, 3340, 6419, 2740, 5540, 300...","[0-11-13-7-5-10-6-15-12-1-3-9-14-2-8-4, 14-12-...","[8, 8, 7, 8, 8, 7, 8, 8, 7, 7, 7, 8, 7, 8, 7, ...","[0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, ...","[0-1-2-3-4-5-6-7, 0-1-2-3-4-5-6-7, 0-1-2-3-4-5...","[0.375, 0.625, 0.7142857142857143, 1.0, 0.5, 1...",working memory,0.987500,0.973333,0.664286,0.875040,"[2024-03-12 18:05:57.112000, 2024-03-12 18:06:...","[2024-03-12 18:06:06.171000, 2024-03-12 18:06:..."
4,FDSA,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1712167480231, 1712167488491, 1712167496271, ...","[1712167482371, 1712167490151, 1712167498131, ...","[0-1-2-3, 0-1-2-3, 0-1-2-3, 0-1-2, 0-1-2-3, 0-...","[2140, 1660, 1860, 2140, 2260, 1440, 1660, 232...","[6-8-15-5-0-1-11-12-7-9-3-10-13-2-4-14, 15-14-...","[4, 4, 4, 3, 4, 3, 3, 4, 3, 3, 4, 4, 3, 4, 4, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, ...","[0-1-2-3, 0-1-2-3, 0-1-2-3, 0-1-2, 0-1-2-3, 0-...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",working memory,0.966667,0.855000,0.639286,0.820317,"[2024-04-03 18:04:40.231000, 2024-04-03 18:04:...","[2024-04-03 18:04:42.371000, 2024-04-03 18:04:..."


In [39]:
df_working_memory['Date'] = df_working_memory['Start_time_Dates'].apply(extract_first_date)

In [40]:
# Assuming our have a DataFrame called df_task_switch
# Rename the "userid" column to "subject_id"
df_working_memory.rename(columns={'userid': 'subject_id'}, inplace=True)

# Rename the "Task_type" column to "game_type"
df_working_memory.rename(columns={'Task_type': 'game_type'}, inplace=True)

In [41]:
df_working_memory.head()

,subject_id,BlockNo,Difficulty,Start_time,End_time,Results_responses,Results_rt,Results_targetvalue_stim,Results_num_stim,Results_correct,Correct Answer,Accuracy,game_type,Accuracy_Easy,Accuracy_Medium,Accuracy_Hard,Accuracy_Overall,Start_time_Dates,End_time_Dates,Date
0,BVCX,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1710353214025, 1710353222205, 1710353229325, ...","[1710353216085, 1710353224405, 1710353231425, ...","[0-1-2-3, 0-1-2-3, 0-1-2, 0-1-2-3, 0-1-2, 0-1-...","[2060, 2200, 2100, 2720, 1980, 2040, 1761, 164...","[5-11-8-13-2-7-1-14-12-3-10-6-15-9-0-4, 7-3-6-...","[4, 4, 3, 4, 3, 4, 3, 3, 3, 3, 4, 4, 4, 4, 4, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0-1-2-3, 0-1-2-3, 0-1-2, 0-1-2-3, 0-1-2, 0-1-...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",working memory,1.000000,0.833333,0.531250,0.788194,"[2024-03-13 18:06:54.025000, 2024-03-13 18:07:...","[2024-03-13 18:06:56.085000, 2024-03-13 18:07:...",2024-03-13
1,CVBN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1710447182643, 1710447189623, 1710447197283, ...","[1710447184703, 1710447191163, 1710447200103, ...","[0-1-2, 0-1-2, 0-1-2-3, 0-1-2, 0-1-2-3, 0-1-2-...","[2060, 1540, 2820, 2120, 2340, 2060, 2660, 242...","[7-4-9-14-15-1-5-12-6-11-0-2-10-8-13-3, 9-5-3-...","[3, 3, 4, 3, 4, 4, 4, 3, 3, 4, 4, 3, 3, 3, 4, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0-1-2, 0-1-2, 0-1-2-3, 0-1-2, 0-1-2-3, 0-1-2-...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",working memory,1.000000,0.896667,0.648214,0.848294,"[2024-03-14 20:13:02.643000, 2024-03-14 20:13:...","[2024-03-14 20:13:04.703000, 2024-03-14 20:13:...",2024-03-14
2,DKLQ,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Medium, Medium, Medium, Medium, Medium, Mediu...","[1707852181317, 1707852191218, 1707852201798, ...","[1707852183897, 1707852194478, 1707852203938, ...","[0-1-2-3-4, 0-1-2-3-4, 0-1-2-3-4, 0-1-2-3-4-5,...","[2580, 3260, 2140, 3179, 2779, 2699, 4420, 278...","[15-10-1-3-4-9-12-5-6-2-13-0-14-11-8-7, 0-3-14...","[5, 5, 5, 6, 6, 5, 5, 6, 5, 6, 5, 6, 5, 5, 6, ...","[1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, ...","[0-1-2-3-4, 0-1-2-3-4, 0-1-2-3-4, 0-1-2-3-4-5,...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6, 1.0, 0.6, ...",working memory,1.000000,0.918333,0.513393,0.810575,"[2024-02-13 19:23:01.317000, 2024-02-13 19:23:...","[2024-02-13 19:23:03.897000, 2024-02-13 19:23:...",2024-02-13
3,EDFR,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Hard, Hard, Hard, Hard, Hard, Hard, Hard, Har...","[1710266757112, 1710266777110, 1710266792850, ...","[1710266766171, 1710266783130, 1710266795630, ...","[0-1-2-5-11-6-7-4, 0-1-2-3-4-15-7-5, 0-1-3-2-4...","[9059, 6020, 2780, 3340, 6419, 2740, 5540, 300...","[0-11-13-7-5-10-6-15-12-1-3-9-14-2-8-4, 14-12-...","[8, 8, 7, 8, 8, 7, 8, 8, 7, 7, 7, 8, 7, 8, 7, ...","[0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, ...","[0-1-2-3-4-5-6-7, 0-1-2-3-4-5-6-7, 0-1-2-3-4-5...","[0.375, 0.625, 0.7142857142857143, 1.0, 0.5, 1...",working memory,0.987500,0.973333,0.664286,0.875040,"[2024-03-12 18:05:57.112000, 2024-03-12 18:06:...","[2024-03-12 18:06:06.171000, 2024-03-12 18:06:...",2024-03-12
4,FDSA,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1712167480231, 1712167488491, 1712167496271, ...","[1712167482371, 1712167490151, 1712167498131, ...","[0-1-2-3, 0-1-2-3, 0-1-2-3, 0-1-2, 0-1-2-3, 0-...","[2140, 1660, 1860, 2140, 2260, 1440, 1660, 232...","[6-8-15-5-0-1-11-12-7-9-3-10-13-2-4-14, 15-14-...","[4, 4, 4, 3, 4, 3, 3, 4, 3, 3, 4, 4, 3, 4, 4, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, ...","[0-1-2-3, 0-1-2-3, 0-1-2-3, 0-1-2, 0-1-2-3, 0-...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",working memory,0.966667,0.855000,0.639286,0.820317,"[2024-04-03 18:04:40.231000, 2024-04-03 18:04:...","[2024-04-03 18:04:42.371000, 2024-04-03 18:04:...",2024-04-03


# Task Switch

In [42]:
# Extract and flatten each dictionary key into separate columns
for key in df_task_switch['data'].iloc[0].keys():
    df_task_switch[key] = df_task_switch['data'].apply(lambda x: extract_dict(x, key))

# Drop the original 'eye_data' column
df_task_switch.drop(columns='data', inplace=True)

In [43]:
df_task_switch.head()

,userid,Blockno,Difficulty,Start_time,End_time,Results_Responses,Results_rt,Results_ind_switch,Results_indtask,Results_trail_target,Correct action,Hit/Miss,Accuracy,Task_type,Accuracy_Easy,Accuracy_Medium,Accuracy_Hard,Accuracy_Overall
0,BVCX,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Hard, Hard, Hard, Hard, Hard, Hard, Hard, Har...","[1710354158650, 1710354160210, 1710354161890, ...","[1710354159530.0, None, None, 1710354164510.0,...","[1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, ...","[880.0, None, None, 940.0, 580.0, 640.0, 700.0...","[0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, ...","[1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, ...","[1, 7, 4, 1, 2, 3, 3, 6, 3, 8, 3, 8, 4, 4, 8, ...","[1, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, ...","[1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, ...","[1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, ...",task switch,0.802469,0.888889,0.469136,0.720165
1,CVBN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Hard, Hard, Hard, Hard, Hard, Hard, Hard, Har...","[1710446510643, 1710446512323, 1710446513843, ...","[None, 1710446513163.0, 1710446514463.0, 17104...","[0, 2, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, ...","[None, 840.0, 620.0, 560.0, 760.0, 520.0, 760....","[0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, ...","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, ...","[4, 8, 3, 2, 1, 9, 8, 1, 1, 4, 3, 4, 4, 1, 4, ...","[2, 2, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 2, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...",task switch,0.962963,0.975309,0.913580,0.950617
2,DKLQ,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1707851511298, 1707851514058, 1707851516058, ...","[1707851513378.0, 1707851515378.0, 17078515166...","[1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, ...","[2080.0, 1320.0, 640.0, 1439.0, 840.0, 600.0, ...","[0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, ...","[0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...","[9, 4, 9, 8, 3, 3, 2, 4, 2, 3, 2, 7, 1, 1, 2, ...","[1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",task switch,0.962963,0.925926,0.728395,0.872428
3,EDFR,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1710265855678, 1710265856858, 1710265858338, ...","[1710265856178.0, 1710265857658.0, 17102658589...","[1, 1, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, ...","[500.0, 800.0, 640.0, 380.0, 920.0, 640.0, 480...","[1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, ...","[0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, ...","[7, 1, 8, 7, 2, 7, 2, 8, 2, 6, 2, 8, 9, 7, 9, ...","[1, 1, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 1, 1, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, ...",task switch,0.925926,0.950617,0.814815,0.897119
4,FDSA,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1712168502445, 1712168504025, 1712168505805, ...","[1712168503345.0, 1712168505125.0, 17121685067...","[2, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, ...","[900.0, 1100.0, 920.0, 1060.0, 680.0, 600.0, 6...","[1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, ...","[0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, ...","[8, 3, 7, 9, 7, 3, 2, 2, 8, 1, 1, 1, 2, 7, 3, ...","[2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, ...",task switch,0.938272,0.790123,0.654321,0.794239


In [44]:
# Create empty lists to store the datetime values for Start_time and End_time
start_time_values = []
end_time_values = []

# Iterate through each row and process the lists of timestamps for both Start_time and End_time
for start_timestamp_list, end_timestamp_list in zip(df_task_switch['Start_time'], df_task_switch['End_time']):
    # Initialize empty lists to store the converted timestamps for this row
    start_row_dates = []
    end_row_dates = []

    # Process each Unix timestamp in the Start_time list and convert to datetime
    for start_timestamp in start_timestamp_list:
        try:
            if start_timestamp is not None:
                start_date = pd.to_datetime(start_timestamp, unit='ms')
                start_row_dates.append(start_date)
        except (ValueError, TypeError):
            # Handle the exception or skip the value if it can't be converted
            pass

    # Process each Unix timestamp in the End_time list and convert to datetime
    for end_timestamp in end_timestamp_list:
        try:
            if end_timestamp is not None:
                end_date = pd.to_datetime(end_timestamp, unit='ms')
                end_row_dates.append(end_date)
        except (ValueError, TypeError):
            # Handle the exception or skip the value if it can't be converted
            pass

    # Append the list of datetime values for Start_time and End_time for this row
    start_time_values.append(start_row_dates)
    end_time_values.append(end_row_dates)

# Create new columns in the DataFrame to store the datetime values for Start_time and End_time
df_task_switch['Start_time_Dates'] = start_time_values
df_task_switch['End_time_Dates'] = end_time_values

In [45]:
# Assuming we have a DataFrame called df_task_switch
# Rename the "userid" column to "subject_id"
df_task_switch.rename(columns={'userid': 'subject_id'}, inplace=True)

# Rename the "Task_type" column to "game_type"
df_task_switch.rename(columns={'Task_type': 'game_type'}, inplace=True)

In [46]:
df_task_switch['Date'] = df_task_switch['Start_time_Dates'].apply(extract_first_date)

In [47]:
df_task_switch.head(12)

,subject_id,Blockno,Difficulty,Start_time,End_time,Results_Responses,Results_rt,Results_ind_switch,Results_indtask,Results_trail_target,...,Hit/Miss,Accuracy,game_type,Accuracy_Easy,Accuracy_Medium,Accuracy_Hard,Accuracy_Overall,Start_time_Dates,End_time_Dates,Date
0,BVCX,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Hard, Hard, Hard, Hard, Hard, Hard, Hard, Har...","[1710354158650, 1710354160210, 1710354161890, ...","[1710354159530.0, None, None, 1710354164510.0,...","[1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, ...","[880.0, None, None, 940.0, 580.0, 640.0, 700.0...","[0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, ...","[1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, ...","[1, 7, 4, 1, 2, 3, 3, 6, 3, 8, 3, 8, 4, 4, 8, ...",...,"[1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, ...","[1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, ...",task switch,0.802469,0.888889,0.469136,0.720165,"[2024-03-13 18:22:38.650000, 2024-03-13 18:22:...","[2024-03-13 18:22:39.529999872, 2024-03-13 18:...",2024-03-13
1,CVBN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Hard, Hard, Hard, Hard, Hard, Hard, Hard, Har...","[1710446510643, 1710446512323, 1710446513843, ...","[None, 1710446513163.0, 1710446514463.0, 17104...","[0, 2, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, ...","[None, 840.0, 620.0, 560.0, 760.0, 520.0, 760....","[0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, ...","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, ...","[4, 8, 3, 2, 1, 9, 8, 1, 1, 4, 3, 4, 4, 1, 4, ...",...,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...",task switch,0.962963,0.975309,0.913580,0.950617,"[2024-03-14 20:01:50.643000, 2024-03-14 20:01:...","[2024-03-14 20:01:53.163000064, 2024-03-14 20:...",2024-03-14
2,DKLQ,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1707851511298, 1707851514058, 1707851516058, ...","[1707851513378.0, 1707851515378.0, 17078515166...","[1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, ...","[2080.0, 1320.0, 640.0, 1439.0, 840.0, 600.0, ...","[0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, ...","[0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...","[9, 4, 9, 8, 3, 3, 2, 4, 2, 3, 2, 7, 1, 1, 2, ...",...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",task switch,0.962963,0.925926,0.728395,0.872428,"[2024-02-13 19:11:51.298000, 2024-02-13 19:11:...","[2024-02-13 19:11:53.377999872, 2024-02-13 19:...",2024-02-13
3,EDFR,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1710265855678, 1710265856858, 1710265858338, ...","[1710265856178.0, 1710265857658.0, 17102658589...","[1, 1, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, ...","[500.0, 800.0, 640.0, 380.0, 920.0, 640.0, 480...","[1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, ...","[0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, ...","[7, 1, 8, 7, 2, 7, 2, 8, 2, 6, 2, 8, 9, 7, 9, ...",...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, ...",task switch,0.925926,0.950617,0.814815,0.897119,"[2024-03-12 17:50:55.678000, 2024-03-12 17:50:...","[2024-03-12 17:50:56.177999872, 2024-03-12 17:...",2024-03-12
4,FDSA,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1712168502445, 1712168504025, 1712168505805, ...","[1712168503345.0, 1712168505125.0, 17121685067...","[2, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, ...","[900.0, 1100.0, 920.0, 1060.0, 680.0, 600.0, 6...","[1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, ...","[0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, ...","[8, 3, 7, 9, 7, 3, 2, 2, 8, 1, 1, 1, 2, 7, 3, ...",...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, ...",task switch,0.938272,0.790123,0.654321,0.794239,"[2024-04-03 18:21:42.445000, 2024-04-03 18:21:...","[2024-04-03 18:21:43.344999936, 2024-04-03 18:...",2024-04-03
5,FG

In [48]:
df_eye.head()

,subject_id,game_type,Timestamp,Gaze_X,Gaze_Y,Pupil_left,Pupil_right,Dates,Date
0,BVCX,baseline,"[1710352642777.923, 1710352642769.658, 1710352...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, 2.95977783203125, 2.990264892578125...","[2024-03-13 17:57:22.777923072, 2024-03-13 17:...",2024-03-13
1,BVCX,enumeration,"[1710354603794.525, 1710354603817.258, 1710354...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2024-03-13 18:30:03.794524928, 2024-03-13 18:...",2024-03-13
2,BVCX,task switch,"[1710354023688.477, 1710354023752.989, 1710354...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2024-03-13 18:20:23.688476928, 2024-03-13 18:...",2024-03-13
3,BVCX,working memory,"[1710353127527.266, 1710353127549.055, 1710353...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2024-03-13 18:05:27.527266048, 2024-03-13 18:...",2024-03-13
4,CVBN,baseline,"[1710444817133.011, 1710444817145.584, 1710444...","[nan, 0.5115212798118591, 0.5106108784675598, ...","[nan, 0.5157326459884644, 0.5222259759902954, ...","[nan, 3.527740478515625, 3.4796142578125, 3.49...","[nan, 3.4715728759765625, 3.43231201171875, 3....","[2024-03-14 19:33:37.133010944, 2024-03-14 19:...",2024-03-14


In [49]:
df_task_switch.head()

,subject_id,Blockno,Difficulty,Start_time,End_time,Results_Responses,Results_rt,Results_ind_switch,Results_indtask,Results_trail_target,...,Hit/Miss,Accuracy,game_type,Accuracy_Easy,Accuracy_Medium,Accuracy_Hard,Accuracy_Overall,Start_time_Dates,End_time_Dates,Date
0,BVCX,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Hard, Hard, Hard, Hard, Hard, Hard, Hard, Har...","[1710354158650, 1710354160210, 1710354161890, ...","[1710354159530.0, None, None, 1710354164510.0,...","[1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, ...","[880.0, None, None, 940.0, 580.0, 640.0, 700.0...","[0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, ...","[1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, ...","[1, 7, 4, 1, 2, 3, 3, 6, 3, 8, 3, 8, 4, 4, 8, ...",...,"[1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, ...","[1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, ...",task switch,0.802469,0.888889,0.469136,0.720165,"[2024-03-13 18:22:38.650000, 2024-03-13 18:22:...","[2024-03-13 18:22:39.529999872, 2024-03-13 18:...",2024-03-13
1,CVBN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Hard, Hard, Hard, Hard, Hard, Hard, Hard, Har...","[1710446510643, 1710446512323, 1710446513843, ...","[None, 1710446513163.0, 1710446514463.0, 17104...","[0, 2, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, ...","[None, 840.0, 620.0, 560.0, 760.0, 520.0, 760....","[0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, ...","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, ...","[4, 8, 3, 2, 1, 9, 8, 1, 1, 4, 3, 4, 4, 1, 4, ...",...,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...",task switch,0.962963,0.975309,0.913580,0.950617,"[2024-03-14 20:01:50.643000, 2024-03-14 20:01:...","[2024-03-14 20:01:53.163000064, 2024-03-14 20:...",2024-03-14
2,DKLQ,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1707851511298, 1707851514058, 1707851516058, ...","[1707851513378.0, 1707851515378.0, 17078515166...","[1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, ...","[2080.0, 1320.0, 640.0, 1439.0, 840.0, 600.0, ...","[0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, ...","[0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...","[9, 4, 9, 8, 3, 3, 2, 4, 2, 3, 2, 7, 1, 1, 2, ...",...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",task switch,0.962963,0.925926,0.728395,0.872428,"[2024-02-13 19:11:51.298000, 2024-02-13 19:11:...","[2024-02-13 19:11:53.377999872, 2024-02-13 19:...",2024-02-13
3,EDFR,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1710265855678, 1710265856858, 1710265858338, ...","[1710265856178.0, 1710265857658.0, 17102658589...","[1, 1, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, ...","[500.0, 800.0, 640.0, 380.0, 920.0, 640.0, 480...","[1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, ...","[0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, ...","[7, 1, 8, 7, 2, 7, 2, 8, 2, 6, 2, 8, 9, 7, 9, ...",...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, ...",task switch,0.925926,0.950617,0.814815,0.897119,"[2024-03-12 17:50:55.678000, 2024-03-12 17:50:...","[2024-03-12 17:50:56.177999872, 2024-03-12 17:...",2024-03-12
4,FDSA,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1712168502445, 1712168504025, 1712168505805, ...","[1712168503345.0, 1712168505125.0, 17121685067...","[2, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, ...","[900.0, 1100.0, 920.0, 1060.0, 680.0, 600.0, 6...","[1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, ...","[0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, ...","[8, 3, 7, 9, 7, 3, 2, 2, 8, 1, 1, 1, 2, 7, 3, ...",...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, ...",task switch,0.938272,0.790123,0.654321,0.794239,"[2024-04-03 18:21:42.445000, 2024-04-03 18:21:...","[2024-04-03 18:21:43.344999936, 2024-04-03 18:...",2024-04-03


In [50]:
'game_type'

'game_type'

In [51]:
df_task_switch['game_type']

0     task switch
1     task switch
2     task switch
3     task switch
4     task switch
         ...     
95    task switch
96    task switch
97    task switch
98    task switch
99    task switch
Name: game_type, Length: 100, dtype: object

In [52]:
len(df_eye)

400

In [53]:
len(df_task_switch)

100

In [54]:
len(df_working_memory)

97

In [55]:
len(df_sensor)

400

In [56]:
df_sensor.head()

,subject_id,game_type,Timestamp,ecg_data,eda_data,Dates,Date
0,BVCX,baseline,"[1710352670.22933, 1710352670.23333, 171035267...","[210.0, 215.0, 210.0, 203.0, 166.0, 132.0, 133...","[209.0, 211.0, 208.0, 201.0, 166.0, 133.0, 131...","[2024-03-13 17:57:50.229330176, 2024-03-13 17:...",2024-03-13
1,BVCX,enumeration,"[1710354601.587721, 1710354601.591721, 1710354...","[236.0, 239.0, 234.0, 236.0, 237.0, 233.0, 239...","[233.0, 236.0, 232.0, 233.0, 235.0, 232.0, 236...","[2024-03-13 18:30:01.587721216, 2024-03-13 18:...",2024-03-13
2,BVCX,task switch,"[1710354051.4275117, 1710354051.4315114, 17103...","[180.0, 178.0, 157.0, 155.0, 153.0, 163.0, 156...","[178.0, 176.0, 155.0, 152.0, 151.0, 160.0, 155...","[2024-03-13 18:20:51.427511808, 2024-03-13 18:...",2024-03-13
3,BVCX,working memory,"[1710353155.3258922, 1710353155.3298922, 17103...","[418.0, 363.0, 272.0, 202.0, 179.0, 157.0, 137...","[416.0, 364.0, 274.0, 203.0, 175.0, 155.0, 137...","[2024-03-13 18:05:55.325892096, 2024-03-13 18:...",2024-03-13
4,CVBN,baseline,"[1710444844.91457, 1710444844.9185698, 1710444...","[207.0, 203.0, 200.0, 196.0, 188.0, 193.0, 186...","[203.0, 202.0, 197.0, 193.0, 187.0, 190.0, 184...","[2024-03-14 19:34:04.914569984, 2024-03-14 19:...",2024-03-14


In [57]:
df_eye.head()

,subject_id,game_type,Timestamp,Gaze_X,Gaze_Y,Pupil_left,Pupil_right,Dates,Date
0,BVCX,baseline,"[1710352642777.923, 1710352642769.658, 1710352...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, 2.95977783203125, 2.990264892578125...","[2024-03-13 17:57:22.777923072, 2024-03-13 17:...",2024-03-13
1,BVCX,enumeration,"[1710354603794.525, 1710354603817.258, 1710354...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2024-03-13 18:30:03.794524928, 2024-03-13 18:...",2024-03-13
2,BVCX,task switch,"[1710354023688.477, 1710354023752.989, 1710354...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2024-03-13 18:20:23.688476928, 2024-03-13 18:...",2024-03-13
3,BVCX,working memory,"[1710353127527.266, 1710353127549.055, 1710353...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2024-03-13 18:05:27.527266048, 2024-03-13 18:...",2024-03-13
4,CVBN,baseline,"[1710444817133.011, 1710444817145.584, 1710444...","[nan, 0.5115212798118591, 0.5106108784675598, ...","[nan, 0.5157326459884644, 0.5222259759902954, ...","[nan, 3.527740478515625, 3.4796142578125, 3.49...","[nan, 3.4715728759765625, 3.43231201171875, 3....","[2024-03-14 19:33:37.133010944, 2024-03-14 19:...",2024-03-14


In [58]:
df_task_switch.head()

,subject_id,Blockno,Difficulty,Start_time,End_time,Results_Responses,Results_rt,Results_ind_switch,Results_indtask,Results_trail_target,...,Hit/Miss,Accuracy,game_type,Accuracy_Easy,Accuracy_Medium,Accuracy_Hard,Accuracy_Overall,Start_time_Dates,End_time_Dates,Date
0,BVCX,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Hard, Hard, Hard, Hard, Hard, Hard, Hard, Har...","[1710354158650, 1710354160210, 1710354161890, ...","[1710354159530.0, None, None, 1710354164510.0,...","[1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, ...","[880.0, None, None, 940.0, 580.0, 640.0, 700.0...","[0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, ...","[1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, ...","[1, 7, 4, 1, 2, 3, 3, 6, 3, 8, 3, 8, 4, 4, 8, ...",...,"[1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, ...","[1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, ...",task switch,0.802469,0.888889,0.469136,0.720165,"[2024-03-13 18:22:38.650000, 2024-03-13 18:22:...","[2024-03-13 18:22:39.529999872, 2024-03-13 18:...",2024-03-13
1,CVBN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Hard, Hard, Hard, Hard, Hard, Hard, Hard, Har...","[1710446510643, 1710446512323, 1710446513843, ...","[None, 1710446513163.0, 1710446514463.0, 17104...","[0, 2, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, ...","[None, 840.0, 620.0, 560.0, 760.0, 520.0, 760....","[0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, ...","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, ...","[4, 8, 3, 2, 1, 9, 8, 1, 1, 4, 3, 4, 4, 1, 4, ...",...,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...",task switch,0.962963,0.975309,0.913580,0.950617,"[2024-03-14 20:01:50.643000, 2024-03-14 20:01:...","[2024-03-14 20:01:53.163000064, 2024-03-14 20:...",2024-03-14
2,DKLQ,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1707851511298, 1707851514058, 1707851516058, ...","[1707851513378.0, 1707851515378.0, 17078515166...","[1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, ...","[2080.0, 1320.0, 640.0, 1439.0, 840.0, 600.0, ...","[0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, ...","[0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...","[9, 4, 9, 8, 3, 3, 2, 4, 2, 3, 2, 7, 1, 1, 2, ...",...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",task switch,0.962963,0.925926,0.728395,0.872428,"[2024-02-13 19:11:51.298000, 2024-02-13 19:11:...","[2024-02-13 19:11:53.377999872, 2024-02-13 19:...",2024-02-13
3,EDFR,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1710265855678, 1710265856858, 1710265858338, ...","[1710265856178.0, 1710265857658.0, 17102658589...","[1, 1, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, ...","[500.0, 800.0, 640.0, 380.0, 920.0, 640.0, 480...","[1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, ...","[0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, ...","[7, 1, 8, 7, 2, 7, 2, 8, 2, 6, 2, 8, 9, 7, 9, ...",...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, ...",task switch,0.925926,0.950617,0.814815,0.897119,"[2024-03-12 17:50:55.678000, 2024-03-12 17:50:...","[2024-03-12 17:50:56.177999872, 2024-03-12 17:...",2024-03-12
4,FDSA,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1712168502445, 1712168504025, 1712168505805, ...","[1712168503345.0, 1712168505125.0, 17121685067...","[2, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, ...","[900.0, 1100.0, 920.0, 1060.0, 680.0, 600.0, 6...","[1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, ...","[0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, ...","[8, 3, 7, 9, 7, 3, 2, 2, 8, 1, 1, 1, 2, 7, 3, ...",...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, ...",task switch,0.938272,0.790123,0.654321,0.794239,"[2024-04-03 18:21:42.445000, 2024-04-03 18:21:...","[2024-04-03 18:21:43.344999936, 2024-04-03 18:...",2024-04-03


In [59]:
df_task_switch["subject_id"].unique()

array(['BVCX', 'CVBN', 'DKLQ', 'EDFR', 'FDSA', 'FGHZ', 'HGFD', 'JHGF',
       'JKLZ', 'MYTZ', 'NMBV', 'NMKL', 'PKJH', 'PLMO', 'QPMN', 'QWER',
       'QWRT', 'QXZW', 'SBQRG', 'SJWQP', 'SVTZM', 'SXVBN', 'TGVB', 'TYDF',
       'WERT', 'WZXE', 'XZPT', 'YUZW', 'ZXAS', 'ZXWE', 'ZYXW', 'BHGY',
       'HJKL', 'JFDE', 'JKLM', 'KJHG', 'LKJH', 'LPDW', 'MJKL', 'MJUY',
       'NBVC', 'NVBX', 'QAXY', 'QPLM', 'REWQ', 'RIQW', 'RTGH', 'SBTBT',
       'SNMVN', 'TYZX', 'UIOP', 'UYGH', 'VIWZ', 'VNPL', 'XCVH', 'XRTY',
       'ZRTY', 'ZXCJ', 'ZXCV', 'ZXNM', 'FTLD', 'FYDR', 'GHJK', 'GTYU',
       'HPLX', 'JKZX', 'LKMN', 'LKPW', 'LPZV', 'NHZA', 'NMJU', 'POIU',
       'STWXJ', 'UJNH', 'UPLK', 'UYTR', 'VBNM', 'XCVN', 'AXSW', 'DFGH',
       'DFZX', 'FDXC', 'FTYP', 'GHFJ', 'LRTB', 'MNBW', 'MNOP', 'MTYH',
       'OPKJ', 'PLRQ', 'QAZW', 'QPOI', 'RQWE', 'SHYXZ', 'SLMJK', 'SPKZR',
       'SRKLP', 'VFTY', 'XUYP', 'YTRE'], dtype=object)

In [60]:
df_sensor["subject_id"].unique()

array(['BVCX', 'CVBN', 'DKLQ', 'EDFR', 'FDSA', 'FGHZ', 'HGFD', 'JHGF',
       'JKLZ', 'MYTZ', 'NMBV', 'NMKL', 'PKJH', 'PLMO', 'QPMN', 'QWER',
       'QWRT', 'QXZW', 'SBQRG', 'SJWQP', 'SVTZM', 'SXVBN', 'TGVB', 'TYDF',
       'WERT', 'WZXE', 'XZPT', 'YUZW', 'ZXAS', 'ZXWE', 'ZYXW', 'BHGY',
       'HJKL', 'JFDE', 'JKLM', 'KJHG', 'LKJH', 'LPDW', 'MJKL', 'MJUY',
       'NBVC', 'NVBX', 'QAXY', 'QPLM', 'REWQ', 'RIQW', 'RTGH', 'SBTBT',
       'SNMVN', 'TYZX', 'UIOP', 'UYGH', 'VIWZ', 'VNPL', 'XCVH', 'XRTY',
       'ZRTY', 'ZXCJ', 'ZXCV', 'ZXNM', 'FTLD', 'FYDR', 'GHJK', 'GTYU',
       'HPLX', 'JKZX', 'LKMN', 'LKPW', 'LPZV', 'NHZA', 'NMJU', 'POIU',
       'STWXJ', 'UJNH', 'UPLK', 'UYTR', 'VBNM', 'XCVN', 'AXSW', 'DFGH',
       'DFZX', 'FDXC', 'FTYP', 'GHFJ', 'LRTB', 'MNBW', 'MNOP', 'MTYH',
       'OPKJ', 'PLRQ', 'QAZW', 'QPOI', 'RQWE', 'SHYXZ', 'SLMJK', 'SPKZR',
       'SRKLP', 'VFTY', 'XUYP', 'YTRE'], dtype=object)

In [61]:
df_task_switch["Date"].unique()

array([datetime.date(2024, 3, 13), datetime.date(2024, 3, 14),
       datetime.date(2024, 2, 13), datetime.date(2024, 3, 12),
       datetime.date(2024, 4, 3), datetime.date(2024, 3, 27),
       datetime.date(2024, 4, 2), datetime.date(2024, 2, 21),
       datetime.date(2024, 3, 6), datetime.date(2024, 1, 30),
       datetime.date(2024, 3, 7), datetime.date(2024, 1, 25),
       datetime.date(2024, 4, 4), datetime.date(2024, 3, 5),
       datetime.date(2024, 3, 26), datetime.date(2024, 1, 22),
       datetime.date(2024, 2, 15), datetime.date(2023, 11, 6),
       datetime.date(2023, 11, 2), datetime.date(2023, 11, 7),
       datetime.date(2024, 4, 9), datetime.date(2024, 2, 27),
       datetime.date(2024, 2, 14), datetime.date(2024, 4, 10),
       datetime.date(2024, 3, 28), datetime.date(2024, 2, 29),
       datetime.date(2023, 10, 31), datetime.date(2024, 4, 11),
       datetime.date(2024, 2, 22), datetime.date(2024, 1, 17),
       datetime.date(2024, 2, 28), datetime.date(2024, 2, 8),

In [62]:
df_sensor["Date"].unique()

array([datetime.date(2024, 3, 13), datetime.date(2024, 3, 14),
       datetime.date(2024, 2, 13), datetime.date(2024, 3, 12),
       datetime.date(2024, 4, 3), datetime.date(2024, 3, 27),
       datetime.date(2024, 4, 2), datetime.date(2024, 2, 21),
       datetime.date(2024, 3, 6), datetime.date(2024, 1, 30),
       datetime.date(2024, 3, 7), datetime.date(2024, 1, 25),
       datetime.date(2024, 4, 4), datetime.date(2024, 3, 5),
       datetime.date(2024, 3, 26), datetime.date(2024, 1, 22),
       datetime.date(2024, 2, 15), datetime.date(2023, 11, 6),
       datetime.date(2023, 11, 2), datetime.date(2023, 11, 7),
       datetime.date(2024, 4, 9), datetime.date(2024, 2, 27),
       datetime.date(2024, 2, 14), datetime.date(2024, 4, 10),
       datetime.date(2024, 3, 28), datetime.date(2024, 2, 29),
       datetime.date(2023, 10, 31), datetime.date(2024, 4, 11),
       datetime.date(2024, 2, 22), datetime.date(2024, 1, 17),
       datetime.date(2024, 2, 28), datetime.date(2024, 2, 8),

In [63]:
df_task_switch["game_type"].unique()

array(['task switch'], dtype=object)

In [77]:
df_enumeration["game_type"].unique()

array(['enumeration'], dtype=object)

In [78]:
df_sensor["game_type"].unique()

array(['baseline', 'enumeration', 'task switch', 'working memory'],
      dtype=object)

# Merging

In [79]:
df_sensor.rename(columns={'Timestamp': 'Timestamp_sensor'}, inplace=True)
df_sensor.rename(columns={'Dates': 'Dates_sensor'}, inplace=True)

In [80]:
df_sensor.head()

,subject_id,game_type,Timestamp_sensor,ecg_data,eda_data,Dates_sensor,Date
0,BVCX,baseline,"[1710352670.22933, 1710352670.23333, 171035267...","[210.0, 215.0, 210.0, 203.0, 166.0, 132.0, 133...","[209.0, 211.0, 208.0, 201.0, 166.0, 133.0, 131...","[2024-03-13 17:57:50.229330176, 2024-03-13 17:...",2024-03-13
1,BVCX,enumeration,"[1710354601.587721, 1710354601.591721, 1710354...","[236.0, 239.0, 234.0, 236.0, 237.0, 233.0, 239...","[233.0, 236.0, 232.0, 233.0, 235.0, 232.0, 236...","[2024-03-13 18:30:01.587721216, 2024-03-13 18:...",2024-03-13
2,BVCX,task switch,"[1710354051.4275117, 1710354051.4315114, 17103...","[180.0, 178.0, 157.0, 155.0, 153.0, 163.0, 156...","[178.0, 176.0, 155.0, 152.0, 151.0, 160.0, 155...","[2024-03-13 18:20:51.427511808, 2024-03-13 18:...",2024-03-13
3,BVCX,working memory,"[1710353155.3258922, 1710353155.3298922, 17103...","[418.0, 363.0, 272.0, 202.0, 179.0, 157.0, 137...","[416.0, 364.0, 274.0, 203.0, 175.0, 155.0, 137...","[2024-03-13 18:05:55.325892096, 2024-03-13 18:...",2024-03-13
4,CVBN,baseline,"[1710444844.91457, 1710444844.9185698, 1710444...","[207.0, 203.0, 200.0, 196.0, 188.0, 193.0, 186...","[203.0, 202.0, 197.0, 193.0, 187.0, 190.0, 184...","[2024-03-14 19:34:04.914569984, 2024-03-14 19:...",2024-03-14


In [81]:
df_eye.head()

,subject_id,game_type,Timestamp,Gaze_X,Gaze_Y,Pupil_left,Pupil_right,Dates,Date
0,BVCX,baseline,"[1710352642777.923, 1710352642769.658, 1710352...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, 2.95977783203125, 2.990264892578125...","[2024-03-13 17:57:22.777923072, 2024-03-13 17:...",2024-03-13
1,BVCX,enumeration,"[1710354603794.525, 1710354603817.258, 1710354...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2024-03-13 18:30:03.794524928, 2024-03-13 18:...",2024-03-13
2,BVCX,task switch,"[1710354023688.477, 1710354023752.989, 1710354...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2024-03-13 18:20:23.688476928, 2024-03-13 18:...",2024-03-13
3,BVCX,working memory,"[1710353127527.266, 1710353127549.055, 1710353...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2024-03-13 18:05:27.527266048, 2024-03-13 18:...",2024-03-13
4,CVBN,baseline,"[1710444817133.011, 1710444817145.584, 1710444...","[nan, 0.5115212798118591, 0.5106108784675598, ...","[nan, 0.5157326459884644, 0.5222259759902954, ...","[nan, 3.527740478515625, 3.4796142578125, 3.49...","[nan, 3.4715728759765625, 3.43231201171875, 3....","[2024-03-14 19:33:37.133010944, 2024-03-14 19:...",2024-03-14


In [82]:
df_eye.rename(columns={'Timestamp': 'Timestamp_eye'}, inplace=True)
df_eye.rename(columns={'Dates': 'Dates_eye'}, inplace=True)

In [83]:
df_eye_sensor = pd.merge(df_eye, df_sensor, on=['subject_id', 'Date', 'game_type'], how='inner')
df_eye_sensor.head()

,subject_id,game_type,Timestamp_eye,Gaze_X,Gaze_Y,Pupil_left,Pupil_right,Dates_eye,Date,Timestamp_sensor,ecg_data,eda_data,Dates_sensor
0,BVCX,baseline,"[1710352642777.923, 1710352642769.658, 1710352...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, 2.95977783203125, 2.990264892578125...","[2024-03-13 17:57:22.777923072, 2024-03-13 17:...",2024-03-13,"[1710352670.22933, 1710352670.23333, 171035267...","[210.0, 215.0, 210.0, 203.0, 166.0, 132.0, 133...","[209.0, 211.0, 208.0, 201.0, 166.0, 133.0, 131...","[2024-03-13 17:57:50.229330176, 2024-03-13 17:..."
1,BVCX,enumeration,"[1710354603794.525, 1710354603817.258, 1710354...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2024-03-13 18:30:03.794524928, 2024-03-13 18:...",2024-03-13,"[1710354601.587721, 1710354601.591721, 1710354...","[236.0, 239.0, 234.0, 236.0, 237.0, 233.0, 239...","[233.0, 236.0, 232.0, 233.0, 235.0, 232.0, 236...","[2024-03-13 18:30:01.587721216, 2024-03-13 18:..."
2,BVCX,task switch,"[1710354023688.477, 1710354023752.989, 1710354...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2024-03-13 18:20:23.688476928, 2024-03-13 18:...",2024-03-13,"[1710354051.4275117, 1710354051.4315114, 17103...","[180.0, 178.0, 157.0, 155.0, 153.0, 163.0, 156...","[178.0, 176.0, 155.0, 152.0, 151.0, 160.0, 155...","[2024-03-13 18:20:51.427511808, 2024-03-13 18:..."
3,BVCX,working memory,"[1710353127527.266, 1710353127549.055, 1710353...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2024-03-13 18:05:27.527266048, 2024-03-13 18:...",2024-03-13,"[1710353155.3258922, 1710353155.3298922, 17103...","[418.0, 363.0, 272.0, 202.0, 179.0, 157.0, 137...","[416.0, 364.0, 274.0, 203.0, 175.0, 155.0, 137...","[2024-03-13 18:05:55.325892096, 2024-03-13 18:..."
4,CVBN,baseline,"[1710444817133.011, 1710444817145.584, 1710444...","[nan, 0.5115212798118591, 0.5106108784675598, ...","[nan, 0.5157326459884644, 0.5222259759902954, ...","[nan, 3.527740478515625, 3.4796142578125, 3.49...","[nan, 3.4715728759765625, 3.43231201171875, 3....","[2024-03-14 19:33:37.133010944, 2024-03-14 19:...",2024-03-14,"[1710444844.91457, 1710444844.9185698, 1710444...","[207.0, 203.0, 200.0, 196.0, 188.0, 193.0, 186...","[203.0, 202.0, 197.0, 193.0, 187.0, 190.0, 184...","[2024-03-14 19:34:04.914569984, 2024-03-14 19:..."


In [84]:
merged_df1 = pd.merge(df_task_switch, df_eye_sensor, on=['subject_id', 'Date', 'game_type'], how='inner')
# Assuming 'Blockno' is the current column name our want to change
merged_df1.rename(columns={'Blockno': 'BlockNo'}, inplace=True)
merged_df1.head()

,subject_id,BlockNo,Difficulty,Start_time,End_time,Results_Responses,Results_rt,Results_ind_switch,Results_indtask,Results_trail_target,...,Timestamp_eye,Gaze_X,Gaze_Y,Pupil_left,Pupil_right,Dates_eye,Timestamp_sensor,ecg_data,eda_data,Dates_sensor
0,BVCX,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Hard, Hard, Hard, Hard, Hard, Hard, Hard, Har...","[1710354158650, 1710354160210, 1710354161890, ...","[1710354159530.0, None, None, 1710354164510.0,...","[1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, ...","[880.0, None, None, 940.0, 580.0, 640.0, 700.0...","[0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, ...","[1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, ...","[1, 7, 4, 1, 2, 3, 3, 6, 3, 8, 3, 8, 4, 4, 8, ...",...,"[1710354023688.477, 1710354023752.989, 1710354...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2024-03-13 18:20:23.688476928, 2024-03-13 18:...","[1710354051.4275117, 1710354051.4315114, 17103...","[180.0, 178.0, 157.0, 155.0, 153.0, 163.0, 156...","[178.0, 176.0, 155.0, 152.0, 151.0, 160.0, 155...","[2024-03-13 18:20:51.427511808, 2024-03-13 18:..."
1,CVBN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Hard, Hard, Hard, Hard, Hard, Hard, Hard, Har...","[1710446510643, 1710446512323, 1710446513843, ...","[None, 1710446513163.0, 1710446514463.0, 17104...","[0, 2, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, ...","[None, 840.0, 620.0, 560.0, 760.0, 520.0, 760....","[0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, ...","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, ...","[4, 8, 3, 2, 1, 9, 8, 1, 1, 4, 3, 4, 4, 1, 4, ...",...,"[1710446344021.539, 1710446344034.475, 1710446...","[nan, 0.4782112240791321, 0.4775261878967285, ...","[nan, 0.1161248981952667, 0.1145759522914886, ...","[nan, 3.11163330078125, 3.08135986328125, 3.15...","[nan, 3.117919921875, 3.0877685546875, 3.14709...","[2024-03-14 19:59:04.021539072, 2024-03-14 19:...","[1710446371.78589, 1710446371.78989, 171044637...","[233.0, 232.0, 227.0, 231.0, 226.0, 227.0, 222...","[230.0, 227.0, 225.0, 229.0, 223.0, 224.0, 219...","[2024-03-14 19:59:31.785890048, 2024-03-14 19:..."
2,DKLQ,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1707851511298, 1707851514058, 1707851516058, ...","[1707851513378.0, 1707851515378.0, 17078515166...","[1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, ...","[2080.0, 1320.0, 640.0, 1439.0, 840.0, 600.0, ...","[0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, ...","[0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...","[9, 4, 9, 8, 3, 3, 2, 4, 2, 3, 2, 7, 1, 1, 2, ...",...,"[1707851364109.044, 1707851364105.267, 1707851...","[nan, 0.2681593894958496, 0.2318558543920517, ...","[nan, 0.5002363324165344, 0.4369730353355407, ...","[nan, 2.40252685546875, 2.5190887451171875, 2....","[nan, 2.5684051513671875, 2.67584228515625, 3....","[2024-02-13 19:09:24.109043968, 2024-02-13 19:...","[1707851391.8103907, 1707851391.814391, 170785...","[212.0, 223.0, 224.0, 222.0, 224.0, 217.0, 196...","[688.0, 690.0, 688.0, 690.0, 688.0, 707.0, 687...","[2024-02-13 19:09:51.810390784, 2024-02-13 19:..."
3,EDFR,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1710265855678, 1710265856858, 1710265858338, ...","[1710265856178.0, 1710265857658.0, 17102658589...","[1, 1, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, ...","[500.0, 800.0, 640.0, 380.0, 920.0, 640.0, 480...","[1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, ...","[0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, ...","[7, 1, 8, 7, 2, 7, 2, 8, 2, 6, 2, 8, 9, 7, 9, ...",...,"[1710265767264.263, 1710265767278.867, 1710265...","[nan, 0.1495557576417923, nan, 0.1168294697999...","[nan, 0.2515062987804413, nan, 0.2056682705879...","[nan, 3.127227783203125, nan, 3.25436401367187...","[nan, 3.293319702148437, nan, 3.39576721191406...","[2024-03-12 17

In [85]:
df_enumeration.head()

,subject_id,BlockNo,Difficulty,Start_time,End_time,Results_Responses,Results_rt,Results_targetvalue,Accuracy,game_type,Accuracy_Easy,Accuracy_Medium,Accuracy_Hard,Accuracy_Overall,Start_time_Dates,End_time_Dates,Date
0,BVCX,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1710354640593, 1710354645123, 1710354648903, ...","[1710354642601, 1710354646364, 1710354650563, ...","[8, 8, 9, 6, 5, 7, 6, 7, 6, 5, 5, 5, 9, 7, 8, ...","[2008, 1241, 1660, 1247, 1428, 1614, 2485, 109...","[9, 8, 9, 6, 5, 7, 6, 7, 6, 5, 5, 5, 9, 7, 8, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",enumeration,0.90,0.55,0.30,0.583333,"[2024-03-13 18:30:40.593000, 2024-03-13 18:30:...","[2024-03-13 18:30:42.601000, 2024-03-13 18:30:...",2024-03-13
1,CVBN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Hard, Hard, Hard, Hard, Hard, Hard, Hard, Har...","[1710445737115, 1710445742795, 1710445747935, ...","[1710445740702, 1710445745849, 1710445753169, ...","[7, 5, 7, 6, 9, 6, 5, 8, 7, 6, 6, 7, 5, 6, 8, ...","[3587, 3054, 5234, 2799, 5034, 1613, 2558, 596...","[7, 5, 7, 6, 9, 6, 5, 8, 8, 8, 6, 8, 5, 6, 9, ...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, ...",enumeration,0.80,0.75,0.70,0.750000,"[2024-03-14 19:48:57.115000, 2024-03-14 19:49:...","[2024-03-14 19:49:00.702000, 2024-03-14 19:49:...",2024-03-14
2,DKLQ,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Medium, Medium, Medium, Medium, Medium, Mediu...","[1707850878427, 1707850882387, 1707850886547, ...","[1707850880091, 1707850884259, 1707850888453, ...","[6, 6, 8, 5, 9, 10, 7, 7, 10, 7, 6, 5, 5, 5, 7...","[1664, 1872, 1906, 1130, 3332, 2137, 1609, 189...","[6, 6, 9, 5, 8, 8, 7, 7, 9, 6, 6, 5, 5, 5, 7, ...","[1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, ...",enumeration,0.90,0.70,0.55,0.716667,"[2024-02-13 19:01:18.427000, 2024-02-13 19:01:...","[2024-02-13 19:01:20.091000, 2024-02-13 19:01:...",2024-02-13
3,EDFR,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1710266348279, 1710266352519, 1710266356759, ...","[1710266349996, 1710266354236, 1710266358444, ...","[5, 6, 9, 5, 6, 7, 8, 7, 8, 9, 6, 7, 5, 6, 7, ...","[1717, 1717, 1685, 1711, 1999, 1412, 1659, 162...","[5, 6, 9, 5, 6, 8, 8, 7, 8, 9, 6, 7, 5, 6, 7, ...","[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",enumeration,0.85,0.80,0.50,0.716667,"[2024-03-12 17:59:08.279000, 2024-03-12 17:59:...","[2024-03-12 17:59:09.996000, 2024-03-12 17:59:...",2024-03-12
4,FDSA,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1712166925090, 1712166931190, 1712166935810, ...","[1712166928655, 1712166933286, 1712166939153, ...","[6, 5, 9, 10, 6, 7, 8, 6, 5, 8, 7, 8, 7, 5, 9,...","[3565, 2096, 3343, 2970, 2511, 2391, 2261, 167...","[6, 5, 9, 8, 7, 6, 8, 6, 5, 9, 7, 8, 7, 5, 9, ...","[1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, ...",enumeration,0.75,0.75,0.65,0.716667,"[2024-04-03 17:55:25.090000, 2024-04-03 17:55:...","[2024-04-03 17:55:28.655000, 2024-04-03 17:55:...",2024-04-03


In [86]:
df_eye_sensor

,subject_id,game_type,Timestamp_eye,Gaze_X,Gaze_Y,Pupil_left,Pupil_right,Dates_eye,Date,Timestamp_sensor,ecg_data,eda_data,Dates_sensor
0,BVCX,baseline,"[1710352642777.923, 1710352642769.658, 1710352...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, 2.95977783203125, 2.990264892578125...","[2024-03-13 17:57:22.777923072, 2024-03-13 17:...",2024-03-13,"[1710352670.22933, 1710352670.23333, 171035267...","[210.0, 215.0, 210.0, 203.0, 166.0, 132.0, 133...","[209.0, 211.0, 208.0, 201.0, 166.0, 133.0, 131...","[2024-03-13 17:57:50.229330176, 2024-03-13 17:..."
1,BVCX,enumeration,"[1710354603794.525, 1710354603817.258, 1710354...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2024-03-13 18:30:03.794524928, 2024-03-13 18:...",2024-03-13,"[1710354601.587721, 1710354601.591721, 1710354...","[236.0, 239.0, 234.0, 236.0, 237.0, 233.0, 239...","[233.0, 236.0, 232.0, 233.0, 235.0, 232.0, 236...","[2024-03-13 18:30:01.587721216, 2024-03-13 18:..."
2,BVCX,task switch,"[1710354023688.477, 1710354023752.989, 1710354...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2024-03-13 18:20:23.688476928, 2024-03-13 18:...",2024-03-13,"[1710354051.4275117, 1710354051.4315114, 17103...","[180.0, 178.0, 157.0, 155.0, 153.0, 163.0, 156...","[178.0, 176.0, 155.0, 152.0, 151.0, 160.0, 155...","[2024-03-13 18:20:51.427511808, 2024-03-13 18:..."
3,BVCX,working memory,"[1710353127527.266, 1710353127549.055, 1710353...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2024-03-13 18:05:27.527266048, 2024-03-13 18:...",2024-03-13,"[1710353155.3258922, 1710353155.3298922, 17103...","[418.0, 363.0, 272.0, 202.0, 179.0, 157.0, 137...","[416.0, 364.0, 274.0, 203.0, 175.0, 155.0, 137...","[2024-03-13 18:05:55.325892096, 2024-03-13 18:..."
4,CVBN,baseline,"[1710444817133.011, 1710444817145.584, 1710444...","[nan, 0.5115212798118591, 0.5106108784675598, ...","[nan, 0.5157326459884644, 0.5222259759902954, ...","[nan, 3.527740478515625, 3.4796142578125, 3.49...","[nan, 3.4715728759765625, 3.43231201171875, 3....","[2024-03-14 19:33:37.133010944, 2024-03-14 19:...",2024-03-14,"[1710444844.91457, 1710444844.9185698, 1710444...","[207.0, 203.0, 200.0, 196.0, 188.0, 193.0, 186...","[203.0, 202.0, 197.0, 193.0, 187.0, 190.0, 184...","[2024-03-14 19:34:04.914569984, 2024-03-14 19:..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,YTRE,baseline,"[1711637695846.3, 1711637695859.122, 171163769...","[nan, 0.6326038837432861, 0.6315056681632996, ...","[nan, 0.9230542778968812, 0.912976622581482, 0...","[nan, 3.5099639892578125, 3.4502410888671875, ...","[nan, 3.8503570556640634, 3.84893798828125, 4....","[2024-03-28 14:54:55.846300160, 2024-03-28 14:...",2024-03-28,"[1711637723.7361555, 1711637723.7401557, 17116...","[144.0, 142.0, 137.0, 133.0, 112.0, 108.0, 106...","[147.0, 140.0, 135.0, 131.0, 111.0, 106.0, 104...","[2024-03-28 14:55:23.736155392, 2024-03-28 14:..."
396,YTRE,enumeration,"[1711638199400.047, 1711638199420.487, 1711638...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2024-03-28 15:03:19.400047104, 2024-03-28 15:...",2024-03-28,"[1711638227.2988112, 1711638227.3028111, 17116...","[199.0, 202.0, 207.0, 197.0, 203.0, 209.0, 213...","[196.0, 200.0, 204.0, 195.0, 198.0, 207.0, 209...","[2024-03-28

In [87]:
merged_df2 = pd.merge(df_working_memory, df_eye_sensor, on=['subject_id', 'Date', 'game_type'], how='inner')
merged_df3 = pd.merge(df_enumeration, df_eye_sensor, on=['subject_id', 'Date', 'game_type'], how='inner')

# Merged df

In [88]:
merged_df1["game_type"].unique()

array(['task switch'], dtype=object)

In [89]:
merged_df2["game_type"].unique()

array(['working memory'], dtype=object)

In [90]:
merged_df3["game_type"].unique()

array(['enumeration'], dtype=object)

In [91]:
# Concatenate merged_df1 and merged_df2
merged_df = pd.concat([merged_df1, merged_df2], ignore_index=True)

# Concatenate merged_df with merged_df3
merged_df = pd.concat([merged_df, merged_df3], ignore_index=True)

In [92]:
merged_df.columns

Index(['subject_id', 'BlockNo', 'Difficulty', 'Start_time', 'End_time',
       'Results_Responses', 'Results_rt', 'Results_ind_switch',
       'Results_indtask', 'Results_trail_target', 'Correct action', 'Hit/Miss',
       'Accuracy', 'game_type', 'Accuracy_Easy', 'Accuracy_Medium',
       'Accuracy_Hard', 'Accuracy_Overall', 'Start_time_Dates',
       'End_time_Dates', 'Date', 'Timestamp_eye', 'Gaze_X', 'Gaze_Y',
       'Pupil_left', 'Pupil_right', 'Dates_eye', 'Timestamp_sensor',
       'ecg_data', 'eda_data', 'Dates_sensor', 'Results_responses',
       'Results_targetvalue_stim', 'Results_num_stim', 'Results_correct',
       'Correct Answer', 'Results_targetvalue'],
      dtype='object')

In [93]:
columns_to_drop = merged_df.filter(like='Result').columns.to_list() + ['Correct action', 'Hit/Miss', 'Accuracy','Correct Answer','BlockNo']
merged_df_copy = merged_df.drop(columns=columns_to_drop)

In [94]:
first_row_difficulty_length = len(merged_df['End_time'].iloc[0])
first_row_difficulty_length

243

In [95]:
merged_df_copy.head()

,subject_id,Difficulty,Start_time,End_time,game_type,Accuracy_Easy,Accuracy_Medium,Accuracy_Hard,Accuracy_Overall,Start_time_Dates,...,Timestamp_eye,Gaze_X,Gaze_Y,Pupil_left,Pupil_right,Dates_eye,Timestamp_sensor,ecg_data,eda_data,Dates_sensor
0,BVCX,"[Hard, Hard, Hard, Hard, Hard, Hard, Hard, Har...","[1710354158650, 1710354160210, 1710354161890, ...","[1710354159530.0, None, None, 1710354164510.0,...",task switch,0.802469,0.888889,0.469136,0.720165,"[2024-03-13 18:22:38.650000, 2024-03-13 18:22:...",...,"[1710354023688.477, 1710354023752.989, 1710354...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2024-03-13 18:20:23.688476928, 2024-03-13 18:...","[1710354051.4275117, 1710354051.4315114, 17103...","[180.0, 178.0, 157.0, 155.0, 153.0, 163.0, 156...","[178.0, 176.0, 155.0, 152.0, 151.0, 160.0, 155...","[2024-03-13 18:20:51.427511808, 2024-03-13 18:..."
1,CVBN,"[Hard, Hard, Hard, Hard, Hard, Hard, Hard, Har...","[1710446510643, 1710446512323, 1710446513843, ...","[None, 1710446513163.0, 1710446514463.0, 17104...",task switch,0.962963,0.975309,0.913580,0.950617,"[2024-03-14 20:01:50.643000, 2024-03-14 20:01:...",...,"[1710446344021.539, 1710446344034.475, 1710446...","[nan, 0.4782112240791321, 0.4775261878967285, ...","[nan, 0.1161248981952667, 0.1145759522914886, ...","[nan, 3.11163330078125, 3.08135986328125, 3.15...","[nan, 3.117919921875, 3.0877685546875, 3.14709...","[2024-03-14 19:59:04.021539072, 2024-03-14 19:...","[1710446371.78589, 1710446371.78989, 171044637...","[233.0, 232.0, 227.0, 231.0, 226.0, 227.0, 222...","[230.0, 227.0, 225.0, 229.0, 223.0, 224.0, 219...","[2024-03-14 19:59:31.785890048, 2024-03-14 19:..."
2,DKLQ,"[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1707851511298, 1707851514058, 1707851516058, ...","[1707851513378.0, 1707851515378.0, 17078515166...",task switch,0.962963,0.925926,0.728395,0.872428,"[2024-02-13 19:11:51.298000, 2024-02-13 19:11:...",...,"[1707851364109.044, 1707851364105.267, 1707851...","[nan, 0.2681593894958496, 0.2318558543920517, ...","[nan, 0.5002363324165344, 0.4369730353355407, ...","[nan, 2.40252685546875, 2.5190887451171875, 2....","[nan, 2.5684051513671875, 2.67584228515625, 3....","[2024-02-13 19:09:24.109043968, 2024-02-13 19:...","[1707851391.8103907, 1707851391.814391, 170785...","[212.0, 223.0, 224.0, 222.0, 224.0, 217.0, 196...","[688.0, 690.0, 688.0, 690.0, 688.0, 707.0, 687...","[2024-02-13 19:09:51.810390784, 2024-02-13 19:..."
3,EDFR,"[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1710265855678, 1710265856858, 1710265858338, ...","[1710265856178.0, 1710265857658.0, 17102658589...",task switch,0.925926,0.950617,0.814815,0.897119,"[2024-03-12 17:50:55.678000, 2024-03-12 17:50:...",...,"[1710265767264.263, 1710265767278.867, 1710265...","[nan, 0.1495557576417923, nan, 0.1168294697999...","[nan, 0.2515062987804413, nan, 0.2056682705879...","[nan, 3.127227783203125, nan, 3.25436401367187...","[nan, 3.293319702148437, nan, 3.39576721191406...","[2024-03-12 17:49:27.264262912, 2024-03-12 17:...","[1710265765.0196905, 1710265765.0236902, 17102...","[188.0, 173.0, 165.0, 159.0, 156.0, 154.0, 153...","[185.0, 173.0, 163.0, 156.0, 155.0, 151.0, 150...","[2024-03-12 17:49:25.019690496, 2024-03-12 17:..."
4,FDSA,"[Easy, Easy, Easy, Easy, Easy, Easy, Easy, Eas...","[1712168502445, 1712168504025, 1712168505805, ...","[1712168503345.0, 1712168505125.0, 17121685067...",task switch,0.938272,0.790123,0.654321,0.794239,"[2024-04-03 18:21:42.445000, 2024-04-03 18:21:...",...,"[1712168380043.668, 1712168380063.884, 1712168...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2024-04-03 18:19:40.043667968, 2024-04-03 18:...","[1712168407.84576, 171216840

In [ ]:
merged_df_copy.to_pickle('merged_df_copy.pkl')

In [ ]:
merged_df_copy["game_type"].unique()

In [ ]:
# Load the CSV file into a DataFrame
merged_df_copy = pd.read_pickle('merged_df_copy.pkl')

In [ ]:
merged_df_copy.head()

In [ ]:
row_index = 0  # Specify the index of the row we want to analyze
selected_row = merged_df_copy.iloc[row_index]

# Iterate over each index and value in the selected row
for column_name, cell_value in selected_row.items():
    # Check if the cell value is a list
    if isinstance(cell_value, list):
        # Get the length of the list and print it
        list_size = len(cell_value)
        print(f"Size of list in column '{column_name}': {list_size}")



In [ ]:
merged_df_copy.iloc[2]

In [ ]:
merged_df_copy.head()

In [ ]:
# Function to interpolate each row separately for gaze data
def interpolate_row(row):
    row['Gaze_X'] = pd.Series(row['Gaze_X']).interpolate().values
    row['Gaze_Y'] = pd.Series(row['Gaze_Y']).interpolate().values
    row['Pupil_left'] = pd.Series(row['Pupil_left']).interpolate().values
    row['Pupil_right'] = pd.Series(row['Pupil_right']).interpolate().values
    return row

# Function to interpolate each row separately for ecg data
def interpolate_row_ecg(row):
    row['ecg_data'] = pd.Series(row['ecg_data']).interpolate().values
    row['Dates_sensor'] = row['Dates_sensor']
    return row

# Initialize empty lists to store the extracted data
extracted_data_1 = []
extracted_data_2 = []

# Iterate through each row in merged_df_copy
for index, (row_index, sample_row) in enumerate(merged_df_copy.iterrows(), 1):
    # Extract the relevant columns
    subject_id = sample_row["subject_id"]
    game_type = sample_row["game_type"]
    difficulty_levels = sample_row["Difficulty"]
    start_times = sample_row["Start_time_Dates"]
    end_times = sample_row["End_time_Dates"]
    pupil_left = sample_row["Pupil_left"]
    pupil_right = sample_row["Pupil_right"]
    ecg_data = sample_row["ecg_data"]
    gaze_x = sample_row["Gaze_X"]
    gaze_y = sample_row["Gaze_Y"]
    dates_eye = sample_row["Dates_eye"]
    dates_sensor = sample_row["Dates_sensor"]

    # Extract unique difficulty levels
    unique_difficulty_levels = set(difficulty_levels)

    # Iterate through each unique difficulty level
    for difficulty_level in unique_difficulty_levels:
        # Find the indices of rows corresponding to the current difficulty level
        indices = [i for i, level in enumerate(difficulty_levels) if level == difficulty_level]

        # Check if there are any rows with this difficulty level
        if not indices:
            continue

        # Find the first occurrence (start time) for this difficulty level
        first_start_time = None
        try:
            if start_times:
                first_start_time = min([pd.to_datetime(start_times[i], errors='coerce') for i in indices if i < len(start_times)])
        except ValueError as e:
            print(f"Error parsing start time for difficulty level {difficulty_level}: {e}")
            continue

        # Find the last occurrence (end time) for this difficulty level
        last_end_time = None
        try:
            if end_times and any(i < len(end_times) for i in indices):
                last_end_time = max([pd.to_datetime(end_times[i], errors='coerce') for i in indices if i < len(end_times)])
        except ValueError as e:
            print(f"Error parsing end time for difficulty level {difficulty_level}: {e}")
            continue

        # Ensure start and end times are not None before proceeding
        if first_start_time is None or last_end_time is None:
            continue

        # Find the indices within the date range
        date_range_indices_eye = [i for i, date in enumerate(dates_eye) if first_start_time <= date <= last_end_time]
        date_range_indices_sensor = [i for i, date in enumerate(dates_sensor) if first_start_time <= date <= last_end_time]

        # Aggregate the data into dictionaries
        aggregated_data_1 = {
            "subject_id": subject_id,
            "game_type": game_type,
            "Difficulty": difficulty_level,
            "Start_time": first_start_time,
            "End_time": last_end_time,
            "Gaze_X": [gaze_x[i] for i in date_range_indices_eye],
            "Gaze_Y": [gaze_y[i] for i in date_range_indices_eye],
            "Dates_eye": [dates_eye[i] for i in date_range_indices_eye],
            "Pupil_left": [pupil_left[i] for i in date_range_indices_eye],
            "Pupil_right": [pupil_right[i] for i in date_range_indices_eye]
        }

        aggregated_data_2 = {
            "subject_id": subject_id,
            "game_type": game_type,
            "Difficulty": difficulty_level,
            "Start_time": first_start_time,
            "End_time": last_end_time,
            "Dates_sensor": [dates_sensor[i] for i in date_range_indices_sensor],
            "ecg_data": [ecg_data[i] for i in date_range_indices_sensor]
        }


        # Append the aggregated data to the respective lists
        extracted_data_1.append(aggregated_data_1)
        extracted_data_2.append(aggregated_data_2)
            
        print(f"Processed chunk {index}/{len(merged_df_copy)}")

# Convert the lists of dictionaries into DataFrames
all_extracted_data_1 = pd.DataFrame(extracted_data_1)
all_extracted_data_2 = pd.DataFrame(extracted_data_2)

# Interpolate each row separately
all_extracted_data_1 = all_extracted_data_1.apply(interpolate_row, axis=1)
all_extracted_data_2 = all_extracted_data_2.apply(interpolate_row_ecg, axis=1)

# Merge the interpolated dataframes
combined_data = pd.merge(all_extracted_data_1, all_extracted_data_2, on=["subject_id", "game_type", "Difficulty", "Start_time", "End_time"])

combined_data.head()


In [ ]:
combined_data['Pupil_right']

In [ ]:
# Calculate missing ratio for Pupil_right
combined_data['Pupil_right_missing_ratio'] = combined_data['Pupil_right'].apply(lambda x: pd.Series(x).isnull().mean())

# Display the DataFrame with the new column
combined_data.head(10)


In [ ]:
# Find the row index with the maximum missing ratio for Pupil_right
max_missing_ratio_index = combined_data['Pupil_right_missing_ratio'].idxmax()

# Get the row with the maximum missing ratio
row_with_max_missing_ratio = combined_data.loc[max_missing_ratio_index]

# Display the row
row_with_max_missing_ratio


In [ ]:
print(row_with_max_missing_ratio['Pupil_right'])


In [ ]:
import matplotlib.pyplot as plt

# Find the row with the maximum missing ratio for Pupil_right
max_missing_ratio_index = combined_data['Pupil_right_missing_ratio'].idxmax()
row_with_max_missing_ratio = combined_data.loc[max_missing_ratio_index]

# Plot the missing values ratio for Pupil_right as a line plot
plt.figure(figsize=(10, 6))
plt.plot(row_with_max_missing_ratio['Pupil_right'], linestyle='-')
plt.title('Missing Values Ratio for Pupil_right')
plt.xlabel('Index')
plt.ylabel('Missing Values Ratio')
plt.grid(True)
plt.show()


In [ ]:
combined_data["game_type"].unique()

In [ ]:
# Choose a random row index
random_row_index = random.randint(0, len(combined_data) - 1)

# Select the random row
random_row = combined_data.iloc[random_row_index]

# Extract the time-series data for plotting
gaze_x = random_row['Gaze_X']
gaze_y = random_row['Gaze_Y']
pupil_left = random_row['Pupil_left']
pupil_right = random_row['Pupil_right']
ecg_data = random_row['ecg_data'][:10000]  # Extract first 1000 values
Dates_eye = random_row['Dates_eye']
Dates_sensor = random_row['Dates_sensor'][:10000]  # Extract first 1000 values
game_type = random_row['game_type']  # Extract game_type

# Plotting
plt.figure(figsize=(12, 10))  # Increase the figure size to accommodate the larger subplot

plt.suptitle(f'Game Type: {game_type}', fontsize=16)  # Set the title for the entire figure

plt.subplot(3, 2, (5, 6))  # Combine the last two subplots into one
plt.plot(Dates_sensor, ecg_data)
plt.title('ECG Data (First 10000 Values)')

plt.subplot(3, 2, 1)
plt.plot(Dates_eye, gaze_x)
plt.title('Gaze_X')

plt.subplot(3, 2, 2)
plt.plot(Dates_eye, gaze_y)
plt.title('Gaze_Y')

plt.subplot(3, 2, 3)
plt.plot(Dates_eye, pupil_left)
plt.title('Pupil_left')

plt.subplot(3, 2, 4)
plt.plot(Dates_eye, pupil_right)
plt.title('Pupil_right')

plt.tight_layout()
plt.show()


In [ ]:
Dates_eye

In [ ]:
from scipy.stats import zscore

# Define a custom function to calculate z-scores while preserving NaN values within each list
def calculate_zscores_with_nan(lst):
    # Convert the list to a NumPy array
    np_array = np.array(lst)
    
    # Check if the array contains any non-numeric values
    if np.issubdtype(np_array.dtype, np.number):
        # Calculate the z-scores while preserving NaN values
        zscores = zscore(np_array, nan_policy='omit')
        return zscores.tolist()
    else:
        # If the array contains non-numeric values, return the original list
        return lst


# Create a new DataFrame 'combined_data_zscore' by applying the custom function to the specified columns
combined_data_zscore = combined_data.copy()  # Make a copy of the original DataFrame

# Apply the custom function to calculate z-scores (preserving NaN values within each list)
combined_data_zscore['Gaze_X'] = combined_data_zscore['Gaze_X'].apply(calculate_zscores_with_nan)
combined_data_zscore['Gaze_Y'] = combined_data_zscore['Gaze_Y'].apply(calculate_zscores_with_nan)
combined_data_zscore['Pupil_left'] = combined_data_zscore['Pupil_left'].apply(calculate_zscores_with_nan)
combined_data_zscore['Pupil_right'] = combined_data_zscore['Pupil_right'].apply(calculate_zscores_with_nan)
combined_data_zscore['ecg_data'] = combined_data_zscore['ecg_data'].apply(calculate_zscores_with_nan)

In [ ]:
# Select the random row
random_row = combined_data_zscore.iloc[random_row_index]

# Choose a random row index
random_row_index = random.randint(0, len(combined_data) - 1)

# Extract the time-series data for plotting
gaze_x = random_row['Gaze_X']
gaze_y = random_row['Gaze_Y']
pupil_left = random_row['Pupil_left']
pupil_right = random_row['Pupil_right']
ecg_data = random_row['ecg_data']  # Extract first 1000 values
Dates_eye = random_row['Dates_eye']
Dates_sensor = random_row['Dates_sensor']

# Plotting
plt.figure(figsize=(12, 10))  # Increase the figure size to accommodate the larger subplot

plt.subplot(3, 2, (5, 6))  # Combine the last two subplots into one
plt.plot(Dates_sensor[:10000], ecg_data[:10000])
plt.title('ECG Data (First 10000 Values)')

plt.subplot(3, 2, 1)
plt.plot(Dates_eye, gaze_x)
plt.title('Gaze_X')

plt.subplot(3, 2, 2)
plt.plot(Dates_eye, gaze_y)
plt.title('Gaze_Y')

plt.subplot(3, 2, 3)
plt.plot(Dates_eye, pupil_left)
plt.title('Pupil_left')

plt.subplot(3, 2, 4)
plt.plot(Dates_eye, pupil_right)
plt.title('Pupil_right')

plt.tight_layout()
plt.show()


In [ ]:
combined_data_zscore['Gaze_X']

In [ ]:
combined_data['Difficulty']

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the 'Difficulty' column
combined_data_zscore['Difficulty_encoded'] = label_encoder.fit_transform(combined_data_zscore['Difficulty'])

# Display the mapping of encoded labels to original labels
print("Encoded labels:", label_encoder.classes_)

# Display the encoded 'Difficulty' column
print(combined_data_zscore['Difficulty_encoded'])


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the 'Difficulty' column
combined_data['Difficulty_encoded'] = label_encoder.fit_transform(combined_data['Difficulty'])

# Display the mapping of encoded labels to original labels
print("Encoded labels:", label_encoder.classes_)

# Display the encoded 'Difficulty' column
print(combined_data['Difficulty_encoded'])


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming combined_data contains our features and target variable (Difficulty level)
# Prepare the data

# X contains our features: gaze_x, gaze_y, pupil_left, pupil_right
# y contains our target variable: difficulty level
X = combined_data_zscore[['Gaze_X', 'Gaze_Y', 'Pupil_left', 'Pupil_right']]
y = combined_data_zscore['Difficulty_encoded']  # Assuming 'Difficulty_encoded' is our target variable

# Split the data into training and test sets
# Here, we allocate 80% of the data for training and 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Check the shape of the resulting datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


In [ ]:
X_train